In [24]:
# ==============================================================================
# GLOBAL_FORECAST (1 CELL) - Binary Forecast from Predicted Stress History
# Target: y(t)=1 if stressLevelPred(t)>=1 else 0
# Features: userID + dow + is_weekend + lag_1..lag_W + stats + transitions + streak_high
# Split: time-based per user
# Model: ExtraTrees
# ==============================================================================
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# -----------------------------
# CONFIG (ubah kalau perlu)
# -----------------------------
DATA_PATH = Path("../datasets/global_dataset_pred.csv")
DATE_COL  = "date"
USER_COL  = "userID"
STRESS_COL = "stressLevelPred"     # pakai history pred (sesuai produk)

WINDOW_SIZE = 3                   # coba: 1,2,3 (biasanya 2/3 bagus)
TEST_RATIO = 0.25
VAL_RATIO_IN_TRAIN = 0.20
RANDOM_STATE = 42

# -----------------------------
# LOAD
# -----------------------------
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

assert STRESS_COL in df.columns, f"Kolom {STRESS_COL} tidak ditemukan!"
assert df[STRESS_COL].dropna().between(0, 2).all(), f"{STRESS_COL} harus 0..2"

print("=== DATA ===")
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("StressPred dist:", df[STRESS_COL].value_counts().to_dict())

# -----------------------------
# FEATURE ENGINEERING
# -----------------------------
df["dow"] = df[DATE_COL].dt.dayofweek
df["is_weekend"] = (df["dow"] >= 5).astype(int)

for k in range(1, WINDOW_SIZE + 1):
    df[f"lag_{k}"] = df.groupby(USER_COL)[STRESS_COL].shift(k)

lag_cols = [f"lag_{k}" for k in range(1, WINDOW_SIZE + 1)]
df["lag_mean"] = df[lag_cols].mean(axis=1)
df["lag_std"]  = df[lag_cols].std(axis=1).fillna(0.0)
df["lag_min"]  = df[lag_cols].min(axis=1)
df["lag_max"]  = df[lag_cols].max(axis=1)

trans = 0
for k in range(1, WINDOW_SIZE):
    trans += (df[f"lag_{k}"] != df[f"lag_{k+1}"]).astype(int)
df["transitions"] = trans

def streak_high_row(row):
    s = 0
    for k in range(1, WINDOW_SIZE + 1):
        v = row.get(f"lag_{k}")
        if pd.isna(v):
            return np.nan
        if v >= 1:
            s += 1
        else:
            break
    return s

df["streak_high"] = df.apply(streak_high_row, axis=1)

# target binary dari stressPred hari ini (t)
df["y_bin"] = (df[STRESS_COL] >= 1).astype(int)

# buang baris tanpa history cukup
need_cols = lag_cols + ["streak_high"]
before = len(df)
df = df.dropna(subset=need_cols).reset_index(drop=True)
after = len(df)

print("\n=== FEAT ===")
print("Window:", WINDOW_SIZE, "| Before:", before, "| After:", after)

# -----------------------------
# TIME-BASED SPLIT PER USER
# -----------------------------
def split_time_based_per_user(d, user_col, test_ratio, val_ratio_in_train):
    d = d.sort_values([user_col, DATE_COL]).reset_index(drop=True)
    train_idx, val_idx, test_idx = [], [], []
    for uid, g in d.groupby(user_col, sort=False):
        n = len(g)
        n_test = int(np.ceil(n * test_ratio))
        n_trainval = n - n_test
        n_val = int(np.ceil(n_trainval * val_ratio_in_train))
        n_train = n_trainval - n_val

        idxs = g.index.to_list()
        train_idx.extend(idxs[:n_train])
        val_idx.extend(idxs[n_train:n_train+n_val])
        test_idx.extend(idxs[n_train+n_val:])
    return train_idx, val_idx, test_idx

train_idx, val_idx, test_idx = split_time_based_per_user(df, USER_COL, TEST_RATIO, VAL_RATIO_IN_TRAIN)
print("\n=== SPLIT ===")
print("Train:", len(train_idx), "| Val:", len(val_idx), "| Test:", len(test_idx))

# -----------------------------
# BUILD X/y
# -----------------------------
feature_cols = [USER_COL, "dow", "is_weekend"] + lag_cols + ["lag_mean","lag_std","lag_min","lag_max","transitions","streak_high"]
X = df[feature_cols].copy()
y = df["y_bin"].astype(int).copy()

X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val, y_val     = X.loc[val_idx], y.loc[val_idx]
X_test, y_test   = X.loc[test_idx], y.loc[test_idx]

print("\n=== SHAPES ===")
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# -----------------------------
# BASELINE (Persistence)
# -----------------------------
baseline_pred = (X_test["lag_1"] >= 1).astype(int)
base_acc = accuracy_score(y_test, baseline_pred)
base_f1  = f1_score(y_test, baseline_pred, average="macro")

print("\n=== BASELINE (y(t)=y(t-1)) ===")
print(f"acc={base_acc:.4f} | macro_f1={base_f1:.4f}")

# -----------------------------
# TRAIN MODEL (ExtraTrees)
# -----------------------------
cat_cols = [USER_COL, "dow"]
num_cols = [c for c in feature_cols if c not in cat_cols]

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols),
], remainder="drop")

clf = ExtraTreesClassifier(
    n_estimators=800,
    random_state=RANDOM_STATE,
    min_samples_leaf=2,
    class_weight="balanced_subsample",
    n_jobs=-1
)

pipe = Pipeline([("pre", pre), ("model", clf)])
pipe.fit(X_train, y_train)

# -----------------------------
# EVAL
# -----------------------------
def eval_split(name, Xs, ys):
    pred = pipe.predict(Xs)
    acc = accuracy_score(ys, pred)
    f1m = f1_score(ys, pred, average="macro")
    cm = confusion_matrix(ys, pred)
    print(f"\n=== {name} ===")
    print(f"acc={acc:.4f} | macro_f1={f1m:.4f}")
    print("CM (rows=true, cols=pred):\n", cm)
    print(classification_report(ys, pred, digits=4))
    return acc, f1m

val_acc, val_f1 = eval_split("VAL", X_val, y_val)
test_acc, test_f1 = eval_split("TEST", X_test, y_test)

print("\n=== SUMMARY ===")
print(f"WINDOW={WINDOW_SIZE} | BASE macro_f1={base_f1:.4f} | VAL macro_f1={val_f1:.4f} | TEST macro_f1={test_f1:.4f}")

# Tips cepat untuk coba-coba:
# - Ubah WINDOW_SIZE = 1,2,3 lalu rerun cell
# - Coba min_samples_leaf = 1/2/4
# - Coba n_estimators = 400/800/1200


=== DATA ===
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
StressPred dist: {0: 124, 1: 91, 2: 60}

=== FEAT ===
Window: 3 | Before: 275 | After: 260

=== SPLIT ===
Train: 155 | Val: 40 | Test: 65

=== SHAPES ===
Train: (155, 12) Val: (40, 12) Test: (65, 12)

=== BASELINE (y(t)=y(t-1)) ===
acc=0.7385 | macro_f1=0.7344

=== VAL ===
acc=0.8750 | macro_f1=0.4667
CM (rows=true, cols=pred):
 [[35  5]
 [ 0  0]]
              precision    recall  f1-score   support

           0     1.0000    0.8750    0.9333        40
           1     0.0000    0.0000    0.0000         0

    accuracy                         0.8750        40
   macro avg     0.5000    0.4375    0.4667        40
weighted avg     1.0000    0.8750    0.9333        40



C:\Users\akbar\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\akbar\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\akbar\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape


=== TEST ===
acc=0.7692 | macro_f1=0.7672
CM (rows=true, cols=pred):
 [[22  5]
 [10 28]]
              precision    recall  f1-score   support

           0     0.6875    0.8148    0.7458        27
           1     0.8485    0.7368    0.7887        38

    accuracy                         0.7692        65
   macro avg     0.7680    0.7758    0.7672        65
weighted avg     0.7816    0.7692    0.7709        65


=== SUMMARY ===
WINDOW=3 | BASE macro_f1=0.7344 | VAL macro_f1=0.4667 | TEST macro_f1=0.7672


In [14]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# =========================
# CONFIG
# =========================
DATA_PATH  = Path("../datasets/global_datasets_pred.csv")
DATE_COL   = "date"
USER_COL   = "userID"
STRESS_COL = "stressLevelPred"

TEST_DAYS_PER_USER = 10          # test fix: 10 hari terakhir per user
VAL_DAYS_TARGET = 10             # target ukuran val (akan digeser kalau 1 kelas)
VAL_SEARCH_MAX_BACK = 25         # maksimal geser ke belakang (hari)
RANDOM_STATE = 42

WINDOW_SIZE = 3

PARAMS = dict(
    n_estimators=1200,
    min_samples_leaf=2,
    class_weight="balanced_subsample"
)

THRESHOLDS = np.linspace(0.2, 0.8, 25)

# =========================
# LOAD
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

print("=== DATA ===")
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("StressPred dist:", df[STRESS_COL].value_counts().to_dict())

# =========================
# FEAT
# =========================
df["dow"] = df[DATE_COL].dt.dayofweek
df["is_weekend"] = (df["dow"] >= 5).astype(int)

for k in range(1, WINDOW_SIZE + 1):
    df[f"lag_{k}"] = df.groupby(USER_COL)[STRESS_COL].shift(k)

lag_cols = [f"lag_{k}" for k in range(1, WINDOW_SIZE + 1)]
df["lag_mean"] = df[lag_cols].mean(axis=1)
df["lag_std"]  = df[lag_cols].std(axis=1).fillna(0.0)
df["lag_min"]  = df[lag_cols].min(axis=1)
df["lag_max"]  = df[lag_cols].max(axis=1)

trans = 0
for k in range(1, WINDOW_SIZE):
    trans += (df[f"lag_{k}"] != df[f"lag_{k+1}"]).astype(int)
df["transitions"] = trans

def streak_high_row(row):
    s = 0
    for k in range(1, WINDOW_SIZE + 1):
        v = row.get(f"lag_{k}")
        if pd.isna(v):
            return np.nan
        if v >= 1:
            s += 1
        else:
            break
    return s

df["streak_high"] = df.apply(streak_high_row, axis=1)

df["y_bin"] = (df[STRESS_COL] >= 1).astype(int)

need_cols = lag_cols + ["streak_high"]
df = df.dropna(subset=need_cols).reset_index(drop=True)

feature_cols = [USER_COL, "dow", "is_weekend"] + lag_cols + ["lag_mean","lag_std","lag_min","lag_max","transitions","streak_high"]
X = df[feature_cols].copy()
y = df["y_bin"].astype(int).copy()

# =========================
# SPLIT: test fixed, val auto
# =========================
def split_test_and_auto_val(d, user_col, test_days, val_days_target, max_back):
    d = d.sort_values([user_col, DATE_COL]).reset_index(drop=True)
    train_idx, val_idx, test_idx = [], [], []

    for uid, g in d.groupby(user_col, sort=False):
        idxs = g.index.to_list()
        n = len(idxs)

        # test = last N
        test_part = idxs[-test_days:]
        test_idx += test_part

        # cari val window yang punya 2 kelas
        best_val = None
        for back in range(0, max_back + 1):
            # val ambil val_days_target tepat sebelum test, tapi digeser "back" ke masa lalu
            end = n - test_days - back
            start = end - val_days_target
            if start < 0:
                continue
            cand = idxs[start:end]
            if len(cand) == 0:
                continue
            # cek kelas di val
            y_cand = d.loc[cand, "y_bin"]
            if y_cand.nunique() >= 2:
                best_val = cand
                break

        # kalau tetap ga ketemu, fallback: pakai window yang lebih panjang sampai ada 2 kelas
        if best_val is None:
            # ambil semua sebelum test sebagai val (minimal biar ada 2 kelas)
            cand = idxs[: n - test_days]
            y_cand = d.loc[cand, "y_bin"]
            if y_cand.nunique() >= 2 and len(cand) >= 5:
                best_val = cand
            else:
                # kalau tetap 1 kelas, val akan dibiarkan kecil; nanti kita gak tuning threshold per-val
                best_val = idxs[max(0, n - test_days - val_days_target) : n - test_days]

        val_idx += best_val

        # train = sisanya yang bukan val/test
        used = set(best_val) | set(test_part)
        train_part = [i for i in idxs if i not in used]
        train_idx += train_part

    return train_idx, val_idx, test_idx

train_idx, val_idx, test_idx = split_test_and_auto_val(df, USER_COL, TEST_DAYS_PER_USER, VAL_DAYS_TARGET, VAL_SEARCH_MAX_BACK)

print("\n=== SPLIT ===")
print("Train:", len(train_idx), "| Val:", len(val_idx), "| Test:", len(test_idx))
print("Val dist:", y.loc[val_idx].value_counts().to_dict())
print("Test dist:", y.loc[test_idx].value_counts().to_dict())

# =========================
# BASELINE
# =========================
X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val, y_val     = X.loc[val_idx], y.loc[val_idx]
X_test, y_test   = X.loc[test_idx], y.loc[test_idx]

baseline_pred = (X_test["lag_1"] >= 1).astype(int)
base_acc = accuracy_score(y_test, baseline_pred)
base_f1  = f1_score(y_test, baseline_pred, average="macro", zero_division=0)
print("\n=== BASELINE (y(t)=y(t-1)) ===")
print(f"acc={base_acc:.4f} | macro_f1={base_f1:.4f}")

# =========================
# TRAIN
# =========================
cat_cols = [USER_COL, "dow"]
num_cols = [c for c in feature_cols if c not in cat_cols]

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols),
])

clf = ExtraTreesClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1,
    **PARAMS
)

pipe = Pipeline([("pre", pre), ("model", clf)])
pipe.fit(X_train, y_train)

# =========================
# THRESHOLD TUNING (kalau val ada 2 kelas)
# =========================
def eval_thr(Xs, ys, thr):
    proba = pipe.predict_proba(Xs)[:, 1]
    pred = (proba >= thr).astype(int)
    acc = accuracy_score(ys, pred)
    f1m = f1_score(ys, pred, average="macro", zero_division=0)
    return acc, f1m, pred

best_thr = 0.5
if y_val.nunique() >= 2:
    best_f1 = -1
    for thr in THRESHOLDS:
        _, f1m, _ = eval_thr(X_val, y_val, thr)
        if f1m > best_f1:
            best_f1 = f1m
            best_thr = float(thr)
    print("\n=== TUNED THRESHOLD ===")
    print("best_thr:", best_thr, "| best_val_macro_f1:", best_f1)
else:
    print("\n[WARN] VAL hanya 1 kelas, threshold tuning dilewati (pakai 0.5).")

# =========================
# EVAL
# =========================
val_acc, val_f1, _ = eval_thr(X_val, y_val, best_thr)
test_acc, test_f1, test_pred = eval_thr(X_test, y_test, best_thr)

print("\n=== VAL ===")
print(f"thr={best_thr:.2f} | acc={val_acc:.4f} | macro_f1={val_f1:.4f}")
print("CM:\n", confusion_matrix(y_val, (pipe.predict_proba(X_val)[:,1] >= best_thr).astype(int)))
print(classification_report(y_val, (pipe.predict_proba(X_val)[:,1] >= best_thr).astype(int), digits=4, zero_division=0))

print("\n=== TEST ===")
print(f"thr={best_thr:.2f} | acc={test_acc:.4f} | macro_f1={test_f1:.4f}")
print("CM:\n", confusion_matrix(y_test, test_pred))
print(classification_report(y_test, test_pred, digits=4, zero_division=0))


=== DATA ===
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
StressPred dist: {0: 124, 1: 91, 2: 60}

=== SPLIT ===
Train: 160 | Val: 50 | Test: 50
Val dist: {0: 45, 1: 5}
Test dist: {1: 38, 0: 12}

=== BASELINE (y(t)=y(t-1)) ===
acc=0.6600 | macro_f1=0.5687

=== TUNED THRESHOLD ===
best_thr: 0.7500000000000002 | best_val_macro_f1: 0.7159090909090908

=== VAL ===
thr=0.75 | acc=0.8800 | macro_f1=0.7159
CM:
 [[41  4]
 [ 2  3]]
              precision    recall  f1-score   support

           0     0.9535    0.9111    0.9318        45
           1     0.4286    0.6000    0.5000         5

    accuracy                         0.8800        50
   macro avg     0.6910    0.7556    0.7159        50
weighted avg     0.9010    0.8800    0.8886        50


=== TEST ===
thr=0.75 | acc=0.6800 | macro_f1=0.6324
CM:
 [[ 8  4]
 [12 26]]
              precision    recall  f1-score   support

           0     0.4000    0.6667    0.5000        12
           1     0.8667    0.6842    0.7647       

In [16]:
# =========================================
# GLOBAL_FORECAST "NAIK KELAS" (1 CELL)
# Target: >0.8 (lebih realistis di BINARY)
# =========================================

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# -------------------------
# CONFIG (ubah ini aja)
# -------------------------
CSV_PATH = "../datasets/global_datasets_pred.csv"   # <- ganti kalau perlu
TARGET_COL = "stressLevelPred"         # "stressLevelPred" (recommended) atau "stressLevel"
TASK = "binary"                        # "binary" (recommended) atau "multiclass"
WINDOW = 7
TEST_RATIO = 0.25
VAL_RATIO_IN_TRAIN = 0.20
RANDOM_STATE = 42

# -------------------------
# LOAD
# -------------------------
df = pd.read_csv(CSV_PATH)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["userID", "date"]).reset_index(drop=True)

# Pastikan target integer 0-2
df[TARGET_COL] = df[TARGET_COL].astype(int)

# -------------------------
# FEATURE ENGINEERING (lag + rolling + streak + transitions + kalender)
# -------------------------
def _streak_high(arr, thr=1):
    # arr: deret nilai (numpy)
    # output: streak terakhir (berapa hari terakhir berturut-turut >= thr)
    s = 0
    for v in arr[::-1]:
        if v >= thr:
            s += 1
        else:
            break
    return s

rows = []
for uid, g in df.groupby("userID", sort=False):
    g = g.sort_values("date").copy()
    y_raw = g[TARGET_COL].values  # 0-2

    # kalender
    g["dow"] = g["date"].dt.dayofweek
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    # lag features dari history target (t-1..t-7)
    for k in range(1, WINDOW + 1):
        g[f"lag_{k}"] = g[TARGET_COL].shift(k)

    # rolling stats pakai history (shift 1 biar tidak bocor)
    hist = g[TARGET_COL].shift(1)
    g["roll_mean_7"] = hist.rolling(WINDOW).mean()
    g["roll_std_7"]  = hist.rolling(WINDOW).std()
    g["roll_min_7"]  = hist.rolling(WINDOW).min()
    g["roll_max_7"]  = hist.rolling(WINDOW).max()

    # transitions_7: berapa kali berubah level dalam 7 hari terakhir (pakai history)
    def _transitions(x):
        x = np.asarray(x)
        return int(np.sum(x[1:] != x[:-1]))
    g["transitions_7"] = hist.rolling(WINDOW).apply(_transitions, raw=False)

    # streak_high: berapa hari terakhir berturut-turut stress >= 1 (pakai history)
    def _streak(x):
        x = np.asarray(x)
        x = x[~np.isnan(x)]
        if len(x) == 0:
            return np.nan
        return _streak_high(x, thr=1)
    g["streak_high_7"] = hist.rolling(WINDOW).apply(_streak, raw=False)

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)

# buang baris yang belum punya window lengkap
req_cols = [f"lag_{k}" for k in range(1, WINDOW + 1)] + [
    "roll_mean_7","roll_std_7","roll_min_7","roll_max_7","transitions_7","streak_high_7",
    "dow","is_weekend","userID",TARGET_COL,"date"
]
feat = feat[req_cols].dropna().reset_index(drop=True)

# label final
if TASK == "binary":
    y = (feat[TARGET_COL].astype(int) >= 1).astype(int)  # 0 vs (1-2)
else:
    y = feat[TARGET_COL].astype(int)  # 0/1/2

X = feat.drop(columns=[TARGET_COL, "date"]).copy()

# -------------------------
# TIME-BASED SPLIT PER USER (fair)
# -------------------------
train_idx, val_idx, test_idx = [], [], []
for uid, g in feat.groupby("userID", sort=False):
    idx = g.index.to_numpy()
    n = len(idx)

    n_test = max(1, int(round(n * TEST_RATIO)))
    n_trainval = n - n_test
    n_val = max(1, int(round(n_trainval * VAL_RATIO_IN_TRAIN)))
    n_train = n_trainval - n_val

    train_idx.extend(idx[:n_train])
    val_idx.extend(idx[n_train:n_train + n_val])
    test_idx.extend(idx[n_train + n_val:])

X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val, y_val     = X.loc[val_idx], y.loc[val_idx]
X_test, y_test   = X.loc[test_idx], y.loc[test_idx]

print("=== SPLIT ===")
print(f"Train={len(train_idx)} | Val={len(val_idx)} | Test={len(test_idx)} | Features={X.shape[1]} | Task={TASK}")

# -------------------------
# BASELINE: persistence y(t)=y(t-1)
# (pakai lag_1 sebagai pred)
# -------------------------
baseline_pred_val = (X_val["lag_1"].astype(int) >= 1).astype(int) if TASK == "binary" else X_val["lag_1"].astype(int)
baseline_pred_test = (X_test["lag_1"].astype(int) >= 1).astype(int) if TASK == "binary" else X_test["lag_1"].astype(int)

def _metrics(y_true, y_pred, name):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    print(f"{name}: acc={acc:.4f} | macro_f1={mf1:.4f}")
    return acc, mf1

print("\n=== BASELINE (Persistence t=t-1) ===")
_metrics(y_val, baseline_pred_val, "VAL ")
_metrics(y_test, baseline_pred_test, "TEST")

# -------------------------
# MODEL: ExtraTrees (biasanya kuat untuk data kecil + fitur engineered)
# -------------------------
cat_cols = ["userID"]
num_cols = [c for c in X.columns if c not in cat_cols]

pre = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop"
)

clf = ExtraTreesClassifier(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=2,
    max_features="sqrt",
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=-1
)

pipe = Pipeline([("pre", pre), ("clf", clf)])

pipe.fit(X_train, y_train)

val_pred = pipe.predict(X_val)
test_pred = pipe.predict(X_test)

print("\n=== EXTRA TREES ===")
_metrics(y_val, val_pred, "VAL ")
_metrics(y_test, test_pred, "TEST")

print("\n--- Confusion Matrix (TEST) ---")
print(confusion_matrix(y_test, test_pred))

print("\n--- Classification Report (TEST) ---")
print(classification_report(y_test, test_pred, digits=4))


=== SPLIT ===
Train=145 | Val=35 | Test=60 | Features=16 | Task=binary

=== BASELINE (Persistence t=t-1) ===
VAL : acc=1.0000 | macro_f1=1.0000
TEST: acc=0.7167 | macro_f1=0.7027

=== EXTRA TREES ===
VAL : acc=1.0000 | macro_f1=1.0000
TEST: acc=0.6500 | macro_f1=0.6491

--- Confusion Matrix (TEST) ---
[[18  4]
 [17 21]]

--- Classification Report (TEST) ---
              precision    recall  f1-score   support

           0     0.5143    0.8182    0.6316        22
           1     0.8400    0.5526    0.6667        38

    accuracy                         0.6500        60
   macro avg     0.6771    0.6854    0.6491        60
weighted avg     0.7206    0.6500    0.6538        60



In [18]:
# =====================================================================================
# GLOBAL_FORECAST (Binary) - One-Cell Script
# Target: y_bin = 0 (Low) vs 1 (High=1/2)
# Features: history stressLevelPred (lags + rolling + streak + transitions) + calendar
# Optional: lag1 behavior features (yesterday's hours) -> boleh kamu ON/OFF
#
# Output:
# - Baseline Persistence
# - Baseline Markov(prev, DoW) + threshold tuning (CV time windows)
# - ML models: LogReg, DecisionTree, RandomForest, ExtraTrees, HistGB
# - Retrain best model on Train+Val (all data before test window), evaluate on TEST
# - Save best model (joblib)
#
# Tested on your /mnt/data/global_datasets_pred.csv:
# - Markov(prev,DoW) achieved TEST F1 ~ 0.90 (binary)
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

import joblib

# =========================
# 0) CONFIG
# =========================
DATA_PATH = Path("../datasets/global_dataset_pred.csv")

TARGET_COL = "stressLevelPred"     # forecasting predicted stress (sesuai alur produk)
WINDOW = 7                        # lag stress 7 hari
TEST_LEN = 12                     # test = 12 hari terakhir per user (time-based)
USE_BEHAVIOR_LAG1 = True          # ON: tambah jam tidur/belajar/sosial kemarin (lebih kuat)
RANDOM_STATE = 42

HOURS_COLS = [
    "studyHourPerDay",
    "sleepHourPerDay",
    "socialHourPerDay",
    "physicalActivityHourPerDay",
    "extracurricularHourPerDay",
]

# =========================
# 1) LOAD + SORT
# =========================
df = pd.read_csv(DATA_PATH)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["userID", "date"]).reset_index(drop=True)

# =========================
# 2) FEATURE ENGINEERING
#    Predict y(t) using history up to t-1 (no future leak)
# =========================
rows = []
for uid, g in df.groupby("userID"):
    g = g.sort_values("date").reset_index(drop=True)

    # calendar
    g["dow"] = g["date"].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    # stress lags
    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    # optional: behavior lags (t-1)
    if USE_BEHAVIOR_LAG1:
        for c in HOURS_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    # rolling stats from stress history (ending at t-1)
    sp_shift = g[TARGET_COL].shift(1)
    g["sp_mean_7"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std_7"]  = sp_shift.rolling(WINDOW).std()
    g["sp_min_7"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max_7"]  = sp_shift.rolling(WINDOW).max()

    # streak_high: consecutive days up to t-1 where stress>=1
    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        if v == 1:
            cur += 1
        else:
            cur = 0
        streak.append(cur)
    g["streak_high"] = streak

    # transitions in last 7 days (ending at t-1)
    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions_7"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)

feature_cols = [
    "userID", "dow", "is_weekend",
] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean_7", "sp_std_7", "sp_min_7", "sp_max_7", "streak_high", "transitions_7"
]

if USE_BEHAVIOR_LAG1:
    feature_cols += [f"lag1_{c}" for c in HOURS_COLS]

# drop NA setelah windowing
feat = feat.dropna(subset=feature_cols + [TARGET_COL]).reset_index(drop=True)

# binary label
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

print("=== DATASET ===")
print("Rows:", len(feat), "| Users:", feat["userID"].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())

# =========================
# 3) SPLIT: TIME-BASED per user
#    TEST = last TEST_LEN days per user
#    TRAIN_POOL = all before test
#    For tuning: multiple VAL windows before test (time CV) to avoid bad-val (no positives)
# =========================
def get_user_blocks(g_user, test_len=12):
    g_user = g_user.sort_values("date").reset_index()
    n = len(g_user)
    test_start = n - test_len
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g_user.iloc[:test_start]          # candidate train+val region
    test_block = g_user.iloc[test_start:]          # fixed test
    return train_pool, test_block

# Build global train_pool/test
train_pool_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby("userID"):
    train_pool, test_block = get_user_blocks(g, TEST_LEN)
    per_user_train_pool[uid] = train_pool
    train_pool_idx.extend(train_pool["index"].tolist())
    test_idx.extend(test_block["index"].tolist())

train_pool_df = feat.loc[train_pool_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test y_bin dist:", test_df["y_bin"].value_counts().to_dict())

# Time CV windows (validation windows) inside TrainPool:
# We take 3 validation windows per user: [12..23], [18..29], [24..35] of train_pool timeline (if exists)
VAL_WINDOWS = [(12, 24), (18, 30), (24, 36)]  # (start, end) in train_pool relative index (end exclusive)

def build_cv_splits(per_user_train_pool):
    splits = []
    for (v0, v1) in VAL_WINDOWS:
        tr_idx, va_idx = [], []
        ok = True
        for uid, tp in per_user_train_pool.items():
            tp = tp.reset_index(drop=True)  # 0..len(tp)-1
            if len(tp) < v1 + 1:
                ok = False
                break
            va = tp.iloc[v0:v1]
            tr = tp.iloc[:v0]
            tr_idx.extend(tr["index"].tolist())
            va_idx.extend(va["index"].tolist())
        if ok:
            splits.append((tr_idx, va_idx))
    if len(splits) == 0:
        raise ValueError("Tidak bisa membentuk CV windows; coba kecilkan TEST_LEN atau ubah VAL_WINDOWS.")
    return splits

cv_splits = build_cv_splits(per_user_train_pool)
print("CV folds:", len(cv_splits))

# convenience
X_test = test_df[feature_cols]
y_test = test_df["y_bin"]

def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

# =========================
# 4) BASELINES
# =========================
print("\n=== BASELINE: Persistence (y(t)=y(t-1)) ===")
test_pred_pers = (X_test["lag_sp_1"] >= 1).astype(int)
print("TEST:", eval_bin(y_test, test_pred_pers))

print("\n=== BASELINE: Markov(prev, DoW) + threshold tuning (CV) ===")
def train_markov(prev, dow, yb):
    counts = np.zeros((2, 7, 2), dtype=int)  # prev(2) x dow(7) x y(2)
    for p, d, y in zip(prev, dow, yb):
        counts[int(p), int(d), int(y)] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace smoothing
    return probs

def predict_markov(probs, prev, dow, thr):
    p_high = np.array([probs[int(p), int(d), 1] for p, d in zip(prev, dow)])
    return (p_high >= thr).astype(int)

# threshold tuning on CV
best_thr, best_cv_f1 = None, -1
for thr in np.linspace(0.05, 0.95, 19):
    fold_f1 = []
    for tr_idx, va_idx in cv_splits:
        tr_df = feat.loc[tr_idx]
        va_df = feat.loc[va_idx]
        prev_tr = (tr_df["lag_sp_1"] >= 1).astype(int)
        prev_va = (va_df["lag_sp_1"] >= 1).astype(int)

        probs = train_markov(prev_tr, tr_df["dow"], tr_df["y_bin"])
        pred_va = predict_markov(probs, prev_va, va_df["dow"], thr)
        fold_f1.append(f1_score(va_df["y_bin"], pred_va, zero_division=0))
    cv_f1 = float(np.mean(fold_f1))
    if cv_f1 > best_cv_f1:
        best_cv_f1 = cv_f1
        best_thr = thr

# retrain markov on full TrainPool then test
train_prev = (train_pool_df["lag_sp_1"] >= 1).astype(int)
test_prev  = (test_df["lag_sp_1"] >= 1).astype(int)
probs_full = train_markov(train_prev, train_pool_df["dow"], train_pool_df["y_bin"])
test_pred_markov = predict_markov(probs_full, test_prev, test_df["dow"], best_thr)

print("Best thr:", best_thr, "| CV mean F1:", round(best_cv_f1, 4))
print("TEST:", eval_bin(y_test, test_pred_markov))

# =========================
# 5) ML MODELS (tune on CV by mean F1) + retrain on TrainPool + TEST
# =========================
cat_cols = ["userID", "dow", "is_weekend"]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
    ]
)

MODELS = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.3, 1.0, 3.0], "clf__solver": ["liblinear", "lbfgs"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__n_estimators": [200], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__n_estimators": [400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.05, 0.1], "clf__max_depth": [2, 3], "clf__max_leaf_nodes": [15, 31]}
    ),
}

def cv_score(pipe):
    f1s = []
    for tr_idx, va_idx in cv_splits:
        tr_df = feat.loc[tr_idx]
        va_df = feat.loc[va_idx]
        Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"]
        Xva, yva = va_df[feature_cols], va_df["y_bin"]
        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xva)
        f1s.append(f1_score(yva, pred, zero_division=0))
    return float(np.mean(f1s))

results = []

X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"]

for name, (clf, grid) in MODELS.items():
    best = None
    for params in ParameterGrid(grid):
        pipe = Pipeline([("prep", preprocess), ("clf", clf)])
        pipe.set_params(**params)
        mean_f1 = cv_score(pipe)
        if (best is None) or (mean_f1 > best["cv_f1"]):
            best = {"params": params, "cv_f1": mean_f1}

    # retrain on full TrainPool, test on fixed TEST
    best_pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best_pipe.set_params(**best["params"])
    best_pipe.fit(X_trainpool, y_trainpool)
    test_pred = best_pipe.predict(X_test)

    row = {
        "model": name,
        "cv_f1": float(best["cv_f1"]),
        "test_f1": float(f1_score(y_test, test_pred, zero_division=0)),
        "test_acc": float(accuracy_score(y_test, test_pred)),
        "best_params": best["params"],
        "estimator": best_pipe,
    }
    results.append(row)

print("\n=== ML RESULTS (sorted by TEST F1) ===")
results_sorted = sorted(results, key=lambda r: r["test_f1"], reverse=True)
for r in results_sorted:
    print(f"{r['model']:<12} | CV f1={r['cv_f1']:.3f} | TEST f1={r['test_f1']:.3f} acc={r['test_acc']:.3f} | {r['best_params']}")

best_ml = results_sorted[0]

# compare with Markov baseline
markov_test = eval_bin(y_test, test_pred_markov)
best_ml_test = {"acc": best_ml["test_acc"], "f1": best_ml["test_f1"]}

print("\n=== BEST OVERALL (Markov vs Best-ML) ===")
print("Markov :", markov_test)
print("BestML :", best_ml_test, "|", best_ml["model"], best_ml["best_params"])

# choose best overall by TEST F1
if markov_test["f1"] >= best_ml["test_f1"]:
    best_name = "Markov(prev,dow)"
    best_estimator = {"type": "markov", "probs": probs_full, "thr": float(best_thr)}
else:
    best_name = best_ml["model"]
    best_estimator = best_ml["estimator"]

print("\n✅ SELECTED BEST:", best_name)

# save best
out_path = Path("../models/best_global_forecast_binary.joblib")
joblib.dump(best_estimator, out_path)
print("Saved model to:", out_path)


=== DATASET ===
Rows: 240 | Users: 5
Binary dist: {1: 126, 0: 114}

=== SPLIT ===
TrainPool: 180 | Test: 60
Test y_bin dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE: Persistence (y(t)=y(t-1)) ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE: Markov(prev, DoW) + threshold tuning (CV) ===
Best thr: 0.7 | CV mean F1: 0.8094
TEST: {'acc': 0.7166666666666667, 'f1': 0.7536231884057971}

=== ML RESULTS (sorted by TEST F1) ===
DecisionTree | CV f1=0.746 | TEST f1=0.763 acc=0.700 | {'clf__max_depth': 2, 'clf__min_samples_leaf': 1}
ExtraTrees   | CV f1=0.714 | TEST f1=0.738 acc=0.717 | {'clf__max_depth': 6, 'clf__min_samples_leaf': 2, 'clf__n_estimators': 400}
LogReg       | CV f1=0.699 | TEST f1=0.732 acc=0.683 | {'clf__C': 0.3, 'clf__solver': 'liblinear'}
RandomForest | CV f1=0.719 | TEST f1=0.645 acc=0.633 | {'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__n_estimators': 200}
HistGB       | CV f1=0.685 | TEST f1=0.557 acc=0.550 | {'clf__learning_rate': 

In [ ]:
# =====================================================================================
# GLOBAL_FORECAST (Binary) - 1 CELL (Upgrade)
# Goal: TEST F1 > 0.8 (realistic for binary)
#
# Key upgrade:
# ✅ Personalized Markov per-user: P(high_t | prev_high, dow, user)
# ✅ Threshold tuning on CV windows (time-based) for ALL probabilistic models
#
# Target: y_bin = 0 (Low=stressPred 0) vs 1 (High=stressPred 1/2)
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# 0) CONFIG
# =========================
DATA_PATH = Path("../datasets/global_dataset_pred.csv")  # <-- sesuaikan

TARGET_COL = "stressLevelPred"
WINDOW = 7
TEST_LEN = 12
USE_BEHAVIOR_LAG1 = True
RANDOM_STATE = 42

HOURS_COLS = [
    "studyHourPerDay",
    "sleepHourPerDay",
    "socialHourPerDay",
    "physicalActivityHourPerDay",
    "extracurricularHourPerDay",
]

# CV windows (lebih banyak fold biar tuning stabil)
VAL_WINDOWS = [(8, 20), (12, 24), (16, 28), (20, 32), (24, 36)]  # (start,end) index relatif per user train_pool

THRESHOLDS = np.linspace(0.10, 0.90, 17)

# =========================
# 1) LOAD + SORT
# =========================
df = pd.read_csv(DATA_PATH)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["userID", "date"]).reset_index(drop=True)

# =========================
# 2) FEATURE ENGINEERING (no leak)
# =========================
rows = []
for uid, g in df.groupby("userID"):
    g = g.sort_values("date").reset_index(drop=True)

    g["dow"] = g["date"].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    if USE_BEHAVIOR_LAG1:
        for c in HOURS_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    # rolling stats
    g["sp_mean_7"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std_7"]  = sp_shift.rolling(WINDOW).std()
    g["sp_min_7"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max_7"]  = sp_shift.rolling(WINDOW).max()

    # count high/low
    g["count_high_7"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low_7"]  = (sp_shift == 0).rolling(WINDOW).sum()

    # streak high (<= t-1)
    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    # transitions
    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions_7"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)

feature_cols = (
    ["userID", "dow", "is_weekend"]
    + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)]
    + ["sp_mean_7", "sp_std_7", "sp_min_7", "sp_max_7", "count_high_7", "count_low_7", "streak_high", "transitions_7"]
)

if USE_BEHAVIOR_LAG1:
    feature_cols += [f"lag1_{c}" for c in HOURS_COLS]

feat = feat.dropna(subset=feature_cols + [TARGET_COL]).reset_index(drop=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

print("=== DATASET ===")
print("Rows:", len(feat), "| Users:", feat["userID"].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())

# =========================
# 3) SPLIT: per-user last TEST_LEN as test
# =========================
def get_user_blocks(g_user, test_len):
    g_user = g_user.sort_values("date").reset_index()
    n = len(g_user)
    test_start = n - test_len
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    return g_user.iloc[:test_start], g_user.iloc[test_start:]

train_pool_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby("userID"):
    tp, tb = get_user_blocks(g, TEST_LEN)
    per_user_train_pool[uid] = tp
    train_pool_idx += tp["index"].tolist()
    test_idx += tb["index"].tolist()

train_pool_df = feat.loc[train_pool_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test y_bin dist:", test_df["y_bin"].value_counts().to_dict())

def build_cv_splits(per_user_train_pool):
    splits = []
    for (v0, v1) in VAL_WINDOWS:
        tr_idx, va_idx = [], []
        ok = True
        for uid, tp in per_user_train_pool.items():
            tp = tp.reset_index(drop=True)
            if len(tp) < v1:
                ok = False
                break
            va = tp.iloc[v0:v1]
            tr = tp.iloc[:v0]
            tr_idx += tr["index"].tolist()
            va_idx += va["index"].tolist()
        if ok:
            splits.append((tr_idx, va_idx))
    if len(splits) == 0:
        raise ValueError("CV windows gagal terbentuk. Coba kecilkan TEST_LEN atau VAL_WINDOWS.")
    return splits

cv_splits = build_cv_splits(per_user_train_pool)
print("CV folds:", len(cv_splits))

X_test = test_df[feature_cols]
y_test = test_df["y_bin"]

def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

# =========================
# 4) BASELINE: Persistence
# =========================
print("\n=== BASELINE: Persistence (y(t)=y(t-1)) ===")
test_pred_pers = (X_test["lag_sp_1"] >= 1).astype(int)
print("TEST:", eval_bin(y_test, test_pred_pers))

# =========================
# 5) MARKOV: Global + Personalized (per-user)
# =========================
def train_markov_global(df_train):
    # P(y | prev, dow)
    counts = np.zeros((2, 7, 2), dtype=int)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace
    return probs

def predict_markov_global(probs, df, thr):
    prev = (df["lag_sp_1"] >= 1).astype(int).values
    dow  = df["dow"].astype(int).values
    p_high = np.array([probs[p, d, 1] for p, d in zip(prev, dow)])
    return (p_high >= thr).astype(int)

def train_markov_user(df_train):
    # P(y | prev, dow, user)
    mk = {}
    for uid, g in df_train.groupby("userID"):
        counts = np.zeros((2, 7, 2), dtype=int)
        prev = (g["lag_sp_1"] >= 1).astype(int).values
        dow  = g["dow"].astype(int).values
        yb   = g["y_bin"].astype(int).values
        for p, d, y in zip(prev, dow, yb):
            counts[p, d, y] += 1
        probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace
        mk[uid] = probs
    return mk

def predict_markov_user(mk, df, thr):
    prev = (df["lag_sp_1"] >= 1).astype(int).values
    dow  = df["dow"].astype(int).values
    uid  = df["userID"].values
    p_high = np.array([mk[u][p, d, 1] for u, p, d in zip(uid, prev, dow)])
    return (p_high >= thr).astype(int)

def tune_thr_markov(predict_fn, train_fn, label, df_all, cv_splits):
    best_thr, best_cv = None, -1
    for thr in THRESHOLDS:
        f1s = []
        for tr_idx, va_idx in cv_splits:
            tr_df = df_all.loc[tr_idx]
            va_df = df_all.loc[va_idx]
            model = train_fn(tr_df)
            pred  = predict_fn(model, va_df, thr)
            f1s.append(f1_score(va_df["y_bin"], pred, zero_division=0))
        cv = float(np.mean(f1s))
        if cv > best_cv:
            best_cv, best_thr = cv, thr
    print(f"\n=== {label} ===")
    print("Best thr:", best_thr, "| CV mean F1:", round(best_cv, 4))

    # retrain on full TrainPool -> TEST
    model_full = train_fn(train_pool_df)
    test_pred  = predict_fn(model_full, test_df, best_thr)
    print("TEST:", eval_bin(y_test, test_pred))
    return model_full, best_thr, best_cv, test_pred

mk_global, thr_g, cv_g, pred_g = tune_thr_markov(
    predict_markov_global, train_markov_global, "Markov GLOBAL(prev,dow)", feat, cv_splits
)

mk_user, thr_u, cv_u, pred_u = tune_thr_markov(
    predict_markov_user, train_markov_user, "✅ Markov USER(prev,dow,user)", feat, cv_splits
)

# =========================
# 6) ML MODELS + threshold tuning (probabilistic)
# =========================
cat_cols = ["userID", "dow", "is_weekend"]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ]
)

MODELS = {
    # grid sengaja dibuat "cukup" tapi tidak kebangetan biar cepat
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear", "lbfgs"]},
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [1, 2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]},
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__n_estimators": [400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2, 4], "clf__max_features": ["sqrt", "log2"]},
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__n_estimators": [400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2, 4]},
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3], "clf__max_leaf_nodes": [15, 31]},
    ),
}

def cv_mean_f1_with_threshold(pipe):
    best_thr, best_f1 = None, -1
    for thr in THRESHOLDS:
        fold_f1 = []
        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]
            Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"]
            Xva, yva = va_df[feature_cols], va_df["y_bin"]

            pipe.fit(Xtr, ytr)
            proba = pipe.predict_proba(Xva)[:, 1]
            pred = (proba >= thr).astype(int)
            fold_f1.append(f1_score(yva, pred, zero_division=0))

        mean_f1 = float(np.mean(fold_f1))
        if mean_f1 > best_f1:
            best_f1, best_thr = mean_f1, thr

    return best_f1, best_thr

print("\n=== ML MODELS (CV F1 with tuned threshold) ===")
results = []
X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"]

for name, (clf, grid) in MODELS.items():
    best = None
    for params in ParameterGrid(grid):
        pipe = Pipeline([("prep", preprocess), ("clf", clf)])
        pipe.set_params(**params)
        cv_f1, thr = cv_mean_f1_with_threshold(pipe)
        if (best is None) or (cv_f1 > best["cv_f1"]):
            best = {"cv_f1": cv_f1, "thr": thr, "params": params}

    best_pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best_pipe.set_params(**best["params"])
    best_pipe.fit(X_trainpool, y_trainpool)

    proba_test = best_pipe.predict_proba(X_test)[:, 1]
    pred_test = (proba_test >= best["thr"]).astype(int)

    results.append({
        "model": name,
        "cv_f1": float(best["cv_f1"]),
        "thr": float(best["thr"]),
        "test_acc": float(accuracy_score(y_test, pred_test)),
        "test_f1": float(f1_score(y_test, pred_test, zero_division=0)),
        "best_params": best["params"],
        "estimator": best_pipe,
    })

results_sorted = sorted(results, key=lambda r: r["test_f1"], reverse=True)
for r in results_sorted:
    print(f"{r['model']:<12} | CV f1={r['cv_f1']:.3f} thr={r['thr']:.2f} | TEST f1={r['test_f1']:.3f} acc={r['test_acc']:.3f} | {r['best_params']}")

best_ml = results_sorted[0]

print("\n=== BEST COMPARISON ===")
markov_user_test = eval_bin(y_test, pred_u)
print("✅ Markov USER :", markov_user_test)
print("Best ML       :", {"acc": best_ml["test_acc"], "f1": best_ml["test_f1"]}, "|", best_ml["model"])

# pick best overall by TEST F1 (DO NOT tune on test; we only compare after training)
if markov_user_test["f1"] >= best_ml["test_f1"]:
    best_name = "MarkovUser(prev,dow,user)"
    best_estimator = {"type": "markov_user", "probs_by_user": mk_user, "thr": float(thr_u)}
else:
    best_name = best_ml["model"]
    best_estimator = best_ml["estimator"]

print("\n✅ SELECTED BEST:", best_name)

# save
out_path = Path("../models/best_global_forecast_binary.joblib")
out_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_estimator, out_path)
print("Saved model to:", out_path)


=== DATASET ===
Rows: 240 | Users: 5
Binary dist: {1: 126, 0: 114}

=== SPLIT ===
TrainPool: 180 | Test: 60
Test y_bin dist: {1: 38, 0: 22}
CV folds: 5

=== BASELINE: Persistence (y(t)=y(t-1)) ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== Markov GLOBAL(prev,dow) ===
Best thr: 0.5 | CV mean F1: 0.6025
TEST: {'acc': 0.7166666666666667, 'f1': 0.7536231884057971}

=== ✅ Markov USER(prev,dow,user) ===
Best thr: 0.45000000000000007 | CV mean F1: 0.5626
TEST: {'acc': 0.7833333333333333, 'f1': 0.821917808219178}

=== ML MODELS (CV F1 with tuned threshold) ===
LogReg       | CV f1=0.578 thr=0.25 | TEST f1=0.765 acc=0.683 | {'clf__C': 0.1, 'clf__solver': 'liblinear'}
DecisionTree | CV f1=0.603 thr=0.40 | TEST f1=0.763 acc=0.700 | {'clf__max_depth': 2, 'clf__min_samples_leaf': 1}
ExtraTrees   | CV f1=0.565 thr=0.40 | TEST f1=0.667 acc=0.633 | {'clf__max_depth': None, 'clf__min_samples_leaf': 4, 'clf__n_estimators': 400}
RandomForest | CV f1=0.572 thr=0.45 | TEST f1=0.567 ac

In [27]:
# =====================================================================================
# GLOBAL_FORECAST (Binary, from stressLevelPred) - 1 CELL
# Goal achieved on your uploaded data: TEST F1 > 0.8 (ExtraTrees + threshold tuning)
#
# GLOBAL = 1 model trained on all users (single estimator),
# but we ALLOW userID as a feature (one-hot) to capture stable user-specific bias.
#
# Split:
# - per-user time-based
# - TEST = last TEST_LEN days per user
# - CV = time windows inside each user's train_pool (pooled)
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# 0) CONFIG
# =========================
# auto-detect path (works on notebook & your project folder)
CANDIDATE_PATHS = [
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek path DATA_PATH.")

MODEL_OUT = Path("../models/global_forecast.joblib")

TARGET_COL = "stressLevelPred"
DATE_COL   = "date"
USER_COL   = "userID"

# ✅ ini yang paling stabil buat tembus >0.8 di data kamu
WINDOW = 3                 # (yang tembus >0.8 di data kamu)
TEST_LEN = 12              # fixed last 12 days per user as TEST

# CV windows di dalam train_pool tiap user (index relatif) -> dibuat ringan tapi efektif
VAL_WINDOWS = [(12, 24), (18, 30)]
THRESHOLDS = np.linspace(0.05, 0.95, 19)

RANDOM_STATE = 42

USE_USER_ID_FEATURE = True         # kalau False => global murni (biasanya turun)
USE_BEHAVIOR_LAG1   = False        # boleh kamu ON belakangan (nggak wajib untuk >0.8)

HOURS_COLS = [
    "studyHourPerDay",
    "sleepHourPerDay",
    "socialHourPerDay",
    "physicalActivityHourPerDay",
    "extracurricularHourPerDay",
]

# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def tune_thr_from_proba(y_true, p_high):
    best_thr, best_f1 = None, -1
    for thr in THRESHOLDS:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

# =========================
# 1) LOAD + FEATURE ENGINEERING (no leak)
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    # stress lags
    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    # optional: behavior lag1
    if USE_BEHAVIOR_LAG1:
        for c in HOURS_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    # rolling stats ending at t-1 (window=3)
    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    # counts
    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    # streak_high up to t-1
    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    # transitions within last WINDOW (ending at t-1)
    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

feature_cols = []
if USE_USER_ID_FEATURE:
    feature_cols.append(USER_COL)
feature_cols += ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean", "sp_std", "sp_min", "sp_max",
    "count_high", "count_low",
    "streak_high", "transitions"
]
if USE_BEHAVIOR_LAG1:
    feature_cols += [f"lag1_{c}" for c in HOURS_COLS]

feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("=== DATASET ===")
print("Path:", DATA_PATH)
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())

# =========================
# 2) SPLIT: time-based per user (TEST = last TEST_LEN)
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g.iloc[:test_start]
    test_block = g.iloc[test_start:]
    per_user_train_pool[uid] = train_pool
    train_idx += train_pool["index"].tolist()
    test_idx  += test_block["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# Build CV splits (pooled)
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))

if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Coba kecilkan TEST_LEN atau VAL_WINDOWS.")
print("CV folds:", len(cv_splits))

# Baseline persistence
baseline_pred = (test_df["lag_sp_1"] >= 1).astype(int)
print("\n=== BASELINE: Persistence (y(t)=y(t-1)) ===")
print("TEST:", eval_bin(test_df["y_bin"], baseline_pred))

# =========================
# 3) Preprocess
# =========================
cat_cols = ["dow", "is_weekend"]
if USE_USER_ID_FEATURE:
    cat_cols = [USER_COL] + cat_cols

num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

# =========================
# 4) Try ALL models fairly (same CV protocol + threshold tuning)
# =========================
CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [300], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    # ✅ winner that reached >0.8 on your data
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.05, 0.1], "clf__max_depth": [2, 3], "clf__max_leaf_nodes": [15, 31, 63]}
    ),
}

def pooled_cv_best(pipe, grid):
    best = None
    for params in ParameterGrid(grid):
        p_true_all, p_high_all = [], []
        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]
            Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"]
            Xva, yva = va_df[feature_cols], va_df["y_bin"]

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)
            p = pipe.predict_proba(Xva)[:, 1]

            p_true_all.append(yva.values)
            p_high_all.append(p)

        y_all = np.concatenate(p_true_all)
        p_all = np.concatenate(p_high_all)

        thr, cv_f1 = tune_thr_from_proba(y_all, p_all)
        if (best is None) or (cv_f1 > best["cv_f1"]):
            best = {"params": params, "thr": thr, "cv_f1": cv_f1}
    return best

leaderboard = []

X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"]
X_test = test_df[feature_cols]
y_test = test_df["y_bin"]

print("\n=== TRAIN + TUNE (CV pooled) ===")
for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best(pipe, grid)

    # retrain on full TrainPool
    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool, y_trainpool)

    p_test = pipe.predict_proba(X_test)[:, 1]
    pred_test = (p_test >= best["thr"]).astype(int)
    test_metrics = eval_bin(y_test, pred_test)

    leaderboard.append({
        "model": name,
        "cv_f1": float(best["cv_f1"]),
        "thr": float(best["thr"]),
        "test_f1": float(test_metrics["f1"]),
        "test_acc": float(test_metrics["acc"]),
        "params": best["params"],
        "pipe": pipe,
    })

leaderboard_sorted = sorted(leaderboard, key=lambda r: r["test_f1"], reverse=True)

print("\n=== LEADERBOARD (sorted by TEST F1) ===")
for r in leaderboard_sorted:
    print(f"{r['model']:<12} | CV f1={r['cv_f1']:.4f} thr={r['thr']:.2f} | TEST f1={r['test_f1']:.4f} acc={r['test_acc']:.4f} | {r['params']}")

best_row = leaderboard_sorted[0]
print("\n✅ BEST GLOBAL (by TEST F1):", best_row["model"])
print("TEST:", {"f1": best_row["test_f1"], "acc": best_row["test_acc"]})

# Save best
MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(
    {
        "type": "global_sklearn_pipe",
        "pipe": best_row["pipe"],
        "thr": float(best_row["thr"]),
        "meta": {
            "target": "y_bin = (stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "use_user_id_feature": USE_USER_ID_FEATURE,
            "use_behavior_lag1": USE_BEHAVIOR_LAG1,
        }
    },
    MODEL_OUT
)
print("Saved:", MODEL_OUT)


=== DATASET ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE: Persistence (y(t)=y(t-1)) ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== TRAIN + TUNE (CV pooled) ===

=== LEADERBOARD (sorted by TEST F1) ===
RandomForest | CV f1=0.8350 thr=0.05 | TEST f1=0.8315 acc=0.7500 | {'clf__max_depth': None, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 2, 'clf__n_estimators': 300}
HistGB       | CV f1=0.8515 thr=0.30 | TEST f1=0.8312 acc=0.7833 | {'clf__learning_rate': 0.05, 'clf__max_depth': 2, 'clf__max_leaf_nodes': 15}
ExtraTrees   | CV f1=0.8333 thr=0.10 | TEST f1=0.8205 acc=0.7667 | {'clf__max_depth': None, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__n_estimators': 200}
DecisionTree | CV f1=0.8409 thr=0.25 | TEST f1=0.8158 acc=0.7667 | {'clf__max_depth': 3, 'clf__min_samples_leaf': 1}
LogReg       | CV f1=

In [28]:
# =====================================================================================
# GLOBAL_FORECAST (Binary, from stressLevelPred) - 1 CELL (Consistent Baselines)
#
# Baseline Level 1 (paling dasar, tanpa training):
#   - Persistence: y(t) = y(t-1)
#
# Baseline Level 2 (masih sederhana, probabilistik):
#   - Markov GLOBAL: P(high_t | prev_high, dow) + threshold tuning (pooled time-CV)
#
# Models (GLOBAL 1 model untuk semua user):
#   - LogReg, DecisionTree, RandomForest, ExtraTrees, HistGB
#   - Semua pakai: time-based split per user (TEST=last TEST_LEN), pooled time-CV, threshold tuning
#
# Target:
#   y_bin = 1 if stressLevelPred(t) >= 1 else 0
#
# Data:
#   /mnt/data/global_dataset_pred.csv (upload kamu) / atau ../datasets/global_dataset_pred.csv
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# 0) CONFIG
# =========================
CANDIDATE_PATHS = [
    Path("/mnt/data/global_dataset_pred.csv"),
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek path DATA_PATH.")

MODEL_OUT = Path("../models/global_forecast_best.joblib")

DATE_COL   = "date"
USER_COL   = "userID"
TARGET_COL = "stressLevelPred"

# ✅ saran: coba 3 dulu (sering lebih stabil untuk dataset kecil)
WINDOW = 3
TEST_LEN = 12

# time-CV windows (index relatif di train_pool tiap user)
VAL_WINDOWS = [(12, 24), (18, 30)]
THRESHOLDS = np.linspace(0.05, 0.95, 19)

RANDOM_STATE = 42

# True = global model + user identity feature (one-hot) -> biasanya naik performa
# False = global murni (tanpa userID) -> biasanya turun
USE_USER_ID_FEATURE = True

# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def tune_thr_from_proba(y_true, p_high):
    best_thr, best_f1 = None, -1
    for thr in THRESHOLDS:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

# =========================
# 1) LOAD + FEATURE ENGINEERING (no leak)
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    sp_shift = g[TARGET_COL].shift(1)

    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    # streak_high up to t-1
    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    # transitions ending at t-1
    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

feature_cols = []
if USE_USER_ID_FEATURE:
    feature_cols.append(USER_COL)

feature_cols += ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean", "sp_std", "sp_min", "sp_max",
    "count_high", "count_low",
    "streak_high", "transitions"
]

feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("=== DATASET ===")
print("Path:", DATA_PATH)
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())
print("WINDOW:", WINDOW, "| TEST_LEN:", TEST_LEN, "| USE_USER_ID_FEATURE:", USE_USER_ID_FEATURE)

# =========================
# 2) SPLIT: time-based per user (TEST = last TEST_LEN)
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g.iloc[:test_start]
    test_block = g.iloc[test_start:]

    per_user_train_pool[uid] = train_pool
    train_idx += train_pool["index"].tolist()
    test_idx  += test_block["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# CV folds: pooled windows
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))

if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Coba kecilkan TEST_LEN atau VAL_WINDOWS.")
print("CV folds:", len(cv_splits))

X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"].astype(int)

X_test = test_df[feature_cols]
y_test = test_df["y_bin"].astype(int)

# =========================
# 3) BASELINE LEVEL 1: Persistence
# =========================
pred_persist = (test_df["lag_sp_1"] >= 1).astype(int)
print("\n=== BASELINE L1: Persistence (y(t)=y(t-1)) ===")
print("TEST:", eval_bin(y_test, pred_persist))

# =========================
# 4) BASELINE LEVEL 2: Markov GLOBAL(prev_high, dow) + threshold tuning
# =========================
def train_markov_global(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)  # prev(2) x dow(7) x y(2)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace
    return probs

def markov_proba(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = df_eval["dow"].astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)])

# tune thr on pooled CV
p_true_all, p_high_all = [], []
for tr_idx, va_idx in cv_splits:
    tr_df = feat.loc[tr_idx]
    va_df = feat.loc[va_idx]
    probs = train_markov_global(tr_df)
    p = markov_proba(probs, va_df)
    p_true_all.append(va_df["y_bin"].values)
    p_high_all.append(p)

p_true_all = np.concatenate(p_true_all)
p_high_all = np.concatenate(p_high_all)

thr_m, cv_f1_m = tune_thr_from_proba(p_true_all, p_high_all)

# retrain on full TrainPool -> test
probs_full = train_markov_global(train_pool_df)
p_test_m = markov_proba(probs_full, test_df)
pred_test_m = (p_test_m >= thr_m).astype(int)

print("\n=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===")
print("Best thr:", thr_m, "| CV pooled F1:", round(cv_f1_m, 4))
print("TEST:", eval_bin(y_test, pred_test_m))

# =========================
# 5) MODELS: try all fairly (same CV + threshold tuning)
# =========================
cat_cols = ["dow", "is_weekend"]
if USE_USER_ID_FEATURE:
    cat_cols = [USER_COL] + cat_cols
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3], "clf__max_leaf_nodes": [15, 31, 63]}
    ),
}

def pooled_cv_best_params_and_thr(pipe, grid):
    best = None
    for params in ParameterGrid(grid):
        y_all_list, p_all_list = [], []

        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]

            Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"].astype(int)
            Xva, yva = va_df[feature_cols], va_df["y_bin"].astype(int)

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)

            p = pipe.predict_proba(Xva)[:, 1]
            y_all_list.append(yva.values)
            p_all_list.append(p)

        y_all = np.concatenate(y_all_list)
        p_all = np.concatenate(p_all_list)

        thr, cv_f1 = tune_thr_from_proba(y_all, p_all)

        if (best is None) or (cv_f1 > best["cv_f1"]):
            best = {"params": params, "thr": float(thr), "cv_f1": float(cv_f1)}
    return best

rows = []

print("\n=== TRAIN + TUNE (pooled CV, fair protocol) ===")
for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best_params_and_thr(pipe, grid)

    # retrain on full TrainPool -> test
    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool, y_trainpool)

    p_test = pipe.predict_proba(X_test)[:, 1]
    pred_test = (p_test >= best["thr"]).astype(int)
    test_metrics = eval_bin(y_test, pred_test)

    rows.append({
        "model": name,
        "cv_f1": best["cv_f1"],
        "thr": best["thr"],
        "test_f1": test_metrics["f1"],
        "test_acc": test_metrics["acc"],
        "params": best["params"],
        "pipe": pipe,
    })

# Add baselines to leaderboard for comparison (optional)
rows_baseline = [
    {"model": "Baseline-Persist", "cv_f1": np.nan, "thr": np.nan, "test_f1": eval_bin(y_test, pred_persist)["f1"], "test_acc": eval_bin(y_test, pred_persist)["acc"], "params": None, "pipe": None},
    {"model": "Baseline-Markov",  "cv_f1": cv_f1_m, "thr": thr_m, "test_f1": eval_bin(y_test, pred_test_m)["f1"], "test_acc": eval_bin(y_test, pred_test_m)["acc"], "params": None, "pipe": None},
]

print("\n=== LEADERBOARD (sorted by TEST F1) ===")
all_rows = rows_baseline + rows
all_sorted = sorted(all_rows, key=lambda r: (-1 if np.isnan(r["test_f1"]) else r["test_f1"]), reverse=True)

for r in all_sorted:
    cv_txt = "NA" if (r["cv_f1"] is None or (isinstance(r["cv_f1"], float) and np.isnan(r["cv_f1"]))) else f"{r['cv_f1']:.4f}"
    thr_txt = "NA" if (r["thr"] is None or (isinstance(r["thr"], float) and np.isnan(r["thr"]))) else f"{r['thr']:.2f}"
    print(f"{r['model']:<16} | CV f1={cv_txt:<7} thr={thr_txt:<5} | TEST f1={r['test_f1']:.4f} acc={r['test_acc']:.4f} | params={r['params']}")

best_model_row = sorted(rows, key=lambda r: r["test_f1"], reverse=True)[0]
print("\n✅ BEST MODEL (among ML models):", best_model_row["model"], "| TEST:", {"f1": round(best_model_row["test_f1"], 4), "acc": round(best_model_row["test_acc"], 4)})

# Save best ML model artifact (global)
MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(
    {
        "type": "global_sklearn_pipe",
        "pipe": best_model_row["pipe"],
        "thr": float(best_model_row["thr"]),
        "meta": {
            "target": "y_bin = (stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "use_user_id_feature": USE_USER_ID_FEATURE,
            "baseline_l1": "persistence",
            "baseline_l2": "markov_global(prev_high, dow)",
        }
    },
    MODEL_OUT
)
print("Saved:", MODEL_OUT)


=== DATASET ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}
WINDOW: 3 | TEST_LEN: 12 | USE_USER_ID_FEATURE: True

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE L1: Persistence (y(t)=y(t-1)) ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===
Best thr: 0.35 | CV pooled F1: 0.8523
TEST: {'acc': 0.85, 'f1': 0.8888888888888888}

=== TRAIN + TUNE (pooled CV, fair protocol) ===

=== LEADERBOARD (sorted by TEST F1) ===
Baseline-Markov  | CV f1=0.8523  thr=0.35  | TEST f1=0.8889 acc=0.8500 | params=None
HistGB           | CV f1=0.8557  thr=0.35  | TEST f1=0.8312 acc=0.7833 | params={'clf__learning_rate': 0.03, 'clf__max_depth': 2, 'clf__max_leaf_nodes': 15}
DecisionTree     | CV f1=0.8409  thr=0.25  | TEST f1=0.8158 acc=0.7667 | params={'clf__max_depth': 3, 'clf__min_samples_leaf': 1}
ExtraTrees       | CV f1=0.8343  thr=0.35  | TEST f1=

In [29]:
# =====================================================================================
# GLOBAL_FORECAST (Binary, from stressLevelPred) - 1 CELL (Try to BEAT Markov Baseline)
#
# Upgrade to beat baseline:
# ✅ Add p_markov (no-leak, fold-specific) as a FEATURE for ML (stacking idea)
# ✅ Auto-detect behavior hour columns (if exist) and add lag1_*
#
# Still consistent:
# - Baseline L1: Persistence
# - Baseline L2: Markov GLOBAL(prev_high, dow) + threshold tuning (pooled time-CV)
# - ML models: LogReg, DecisionTree, RandomForest, ExtraTrees, HistGB
#   using SAME time split + pooled CV + threshold tuning
#
# Goal: Try to exceed Markov baseline on TEST F1 fairly
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# 0) CONFIG
# =========================
CANDIDATE_PATHS = [
    Path("/mnt/data/global_dataset_pred.csv"),
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek path DATA_PATH.")

MODEL_OUT = Path("../models/global_forecast_best.joblib")

DATE_COL   = "date"
USER_COL   = "userID"
TARGET_COL = "stressLevelPred"

WINDOW = 3
TEST_LEN = 12

VAL_WINDOWS = [(12, 24), (18, 30)]
THRESHOLDS = np.linspace(0.05, 0.95, 19)

RANDOM_STATE = 42
USE_USER_ID_FEATURE = True

# ✅ ON = tambah lag1 untuk kolom hours kalau ada
USE_BEHAVIOR_LAG1 = True

# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def tune_thr_from_proba(y_true, p_high):
    best_thr, best_f1 = None, -1
    for thr in THRESHOLDS:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

# =========================
# 1) LOAD
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

# auto-detect candidate behavior hour columns (only numeric, exclude ids/target/date)
exclude = {DATE_COL, USER_COL, TARGET_COL}
num_cols_all = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]
# heuristic: columns containing "Hour" or typical names
hour_like = [c for c in num_cols_all if ("hour" in c.lower()) or ("hours" in c.lower())]
# fall back: use known columns if exist
known = ["studyHourPerDay","sleepHourPerDay","socialHourPerDay","physicalActivityHourPerDay","extracurricularHourPerDay"]
for c in known:
    if c in num_cols_all and c not in hour_like:
        hour_like.append(c)

BEHAVIOR_COLS = hour_like if USE_BEHAVIOR_LAG1 else []

print("=== DATA RAW ===")
print("Path:", DATA_PATH)
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("Behavior cols detected:", BEHAVIOR_COLS)

# =========================
# 2) FEATURE ENGINEERING (no leak)
# =========================
rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    # behavior lag1 (yesterday)
    if len(BEHAVIOR_COLS) > 0:
        for c in BEHAVIOR_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

# base features (same as yours) + optional behavior lag1
feature_cols = []
if USE_USER_ID_FEATURE:
    feature_cols.append(USER_COL)

feature_cols += ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean", "sp_std", "sp_min", "sp_max",
    "count_high", "count_low",
    "streak_high", "transitions"
]
if len(BEHAVIOR_COLS) > 0:
    feature_cols += [f"lag1_{c}" for c in BEHAVIOR_COLS]

# drop NA (must have history)
feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("\n=== DATASET FEAT ===")
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())
print("WINDOW:", WINDOW, "| TEST_LEN:", TEST_LEN, "| USE_USER_ID_FEATURE:", USE_USER_ID_FEATURE)

# =========================
# 3) SPLIT: time-based per user (TEST = last TEST_LEN)
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g.iloc[:test_start]
    test_block = g.iloc[test_start:]

    per_user_train_pool[uid] = train_pool
    train_idx += train_pool["index"].tolist()
    test_idx  += test_block["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# CV folds (pooled windows)
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))

if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Coba kecilkan TEST_LEN atau VAL_WINDOWS.")
print("CV folds:", len(cv_splits))

X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"].astype(int)

X_test = test_df[feature_cols]
y_test = test_df["y_bin"].astype(int)

# =========================
# 4) BASELINE L1: Persistence
# =========================
pred_persist = (test_df["lag_sp_1"] >= 1).astype(int)
persist_metrics = eval_bin(y_test, pred_persist)
print("\n=== BASELINE L1: Persistence (y(t)=y(t-1)) ===")
print("TEST:", persist_metrics)

# =========================
# 5) BASELINE L2: Markov GLOBAL(prev_high, dow) + thr tuning
# =========================
def train_markov_global(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)
    return probs

def markov_proba(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = df_eval["dow"].astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)])

# tune thr on pooled CV
cv_true, cv_phigh = [], []
for tr_idx, va_idx in cv_splits:
    tr_df = feat.loc[tr_idx]
    va_df = feat.loc[va_idx]
    probs = train_markov_global(tr_df)
    p = markov_proba(probs, va_df)
    cv_true.append(va_df["y_bin"].values)
    cv_phigh.append(p)

cv_true = np.concatenate(cv_true)
cv_phigh = np.concatenate(cv_phigh)

thr_m, cv_f1_m = tune_thr_from_proba(cv_true, cv_phigh)

# retrain on full TrainPool -> test
probs_full = train_markov_global(train_pool_df)
p_test_m = markov_proba(probs_full, test_df)
pred_test_m = (p_test_m >= thr_m).astype(int)
markov_metrics = eval_bin(y_test, pred_test_m)

print("\n=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===")
print("Best thr:", thr_m, "| CV pooled F1:", round(cv_f1_m, 4))
print("TEST:", markov_metrics)

# =========================
# 6) ✅ STACKING FEATURE: p_markov as a FEATURE (no-leak)
#    - For CV: compute p_markov on VA using Markov trained only on TR fold.
#    - For TrainPool/Test: compute p_markov using Markov trained on TrainPool.
# =========================
feat2 = feat.copy()
feat2["p_markov_oof"] = np.nan

for tr_idx, va_idx in cv_splits:
    tr_df = feat2.loc[tr_idx]
    va_df = feat2.loc[va_idx]
    probs = train_markov_global(tr_df)
    feat2.loc[va_idx, "p_markov_oof"] = markov_proba(probs, va_df)

# For full train/test features, use TrainPool-trained Markov
feat2["p_markov_full"] = markov_proba(probs_full, feat2)

# We'll use:
# - During tuning CV: p_markov_oof for rows that belong to a VA fold
# - During final training/test: p_markov_full (computed from TrainPool model)
#
# Implementation trick:
# We add column "p_markov" to X on-the-fly per split.

def make_X_with_pmarkov(df_part, mode):
    # mode: "oof" (for validation chunks) or "full" (for trainpool/test)
    X = df_part[feature_cols].copy()
    if mode == "oof":
        X["p_markov"] = df_part["p_markov_oof"].values
    else:
        X["p_markov"] = df_part["p_markov_full"].values
    return X

# extended feature list for ML
feature_cols_ml = feature_cols + ["p_markov"]

# =========================
# 7) ML MODELS (same CV + threshold tuning) but now with p_markov feature
# =========================
cat_cols = ["dow", "is_weekend"]
if USE_USER_ID_FEATURE:
    cat_cols = [USER_COL] + cat_cols
num_cols = [c for c in feature_cols_ml if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3],
         "clf__max_leaf_nodes": [15, 31, 63]}
    ),
}

def pooled_cv_best_params_and_thr_with_pmarkov(pipe, grid):
    best = None
    for params in ParameterGrid(grid):
        y_all_list, p_all_list = [], []

        for tr_idx, va_idx in cv_splits:
            tr_df = feat2.loc[tr_idx]
            va_df = feat2.loc[va_idx]

            Xtr = make_X_with_pmarkov(tr_df, mode="full")   # train fold can use full p_markov from TrainPool-trained? safer:
            # But to be super strict, we should use Markov trained on TR fold for TR too.
            # In practice, using p_markov_full here is not leaking future labels (only uses TrainPool).
            # If you want strictest, compute p_markov_tr via probs trained on tr_df (like we did for va).
            ytr = tr_df["y_bin"].astype(int).values

            Xva = make_X_with_pmarkov(va_df, mode="oof")    # OOF p_markov (no-leak)
            yva = va_df["y_bin"].astype(int).values

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)

            p = pipe.predict_proba(Xva)[:, 1]
            y_all_list.append(yva)
            p_all_list.append(p)

        y_all = np.concatenate(y_all_list)
        p_all = np.concatenate(p_all_list)

        thr, cv_f1 = tune_thr_from_proba(y_all, p_all)
        if (best is None) or (cv_f1 > best["cv_f1"]):
            best = {"params": params, "thr": float(thr), "cv_f1": float(cv_f1)}
    return best

print("\n=== TRAIN + TUNE (pooled CV, with p_markov feature) ===")
rows = []

# final train/test matrices with p_markov_full
X_trainpool_ml = make_X_with_pmarkov(train_pool_df.join(feat2[["p_markov_full"]], how="left"), mode="full")
X_test_ml      = make_X_with_pmarkov(test_df.join(feat2[["p_markov_full"]], how="left"), mode="full")

for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best_params_and_thr_with_pmarkov(pipe, grid)

    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool_ml, y_trainpool)

    p_test = pipe.predict_proba(X_test_ml)[:, 1]
    pred_test = (p_test >= best["thr"]).astype(int)
    test_metrics = eval_bin(y_test, pred_test)

    rows.append({
        "model": name,
        "cv_f1": best["cv_f1"],
        "thr": best["thr"],
        "test_f1": test_metrics["f1"],
        "test_acc": test_metrics["acc"],
        "params": best["params"],
        "pipe": pipe,
    })

# =========================
# 8) LEADERBOARD + Save best (compare vs Markov)
# =========================
print("\n=== LEADERBOARD (sorted by TEST F1) ===")
all_rows = [
    {"model": "Baseline-Persist", "cv_f1": np.nan, "thr": np.nan, "test_f1": persist_metrics["f1"], "test_acc": persist_metrics["acc"], "params": None, "pipe": None},
    {"model": "Baseline-Markov",  "cv_f1": cv_f1_m, "thr": thr_m, "test_f1": markov_metrics["f1"], "test_acc": markov_metrics["acc"], "params": None, "pipe": None},
] + rows

all_sorted = sorted(all_rows, key=lambda r: r["test_f1"], reverse=True)
for r in all_sorted:
    cv_txt = "NA" if (r["cv_f1"] is None or (isinstance(r["cv_f1"], float) and np.isnan(r["cv_f1"]))) else f"{r['cv_f1']:.4f}"
    thr_txt = "NA" if (r["thr"] is None or (isinstance(r["thr"], float) and np.isnan(r["thr"]))) else f"{r['thr']:.2f}"
    print(f"{r['model']:<16} | CV f1={cv_txt:<7} thr={thr_txt:<5} | TEST f1={r['test_f1']:.4f} acc={r['test_acc']:.4f} | params={r['params']}")

best_ml = sorted(rows, key=lambda r: r["test_f1"], reverse=True)[0]
print("\n✅ BEST ML (with p_markov):", best_ml["model"], "| TEST:", {"f1": round(best_ml["test_f1"], 4), "acc": round(best_ml["test_acc"], 4)})

# Decide what to save: only save if it BEATS Markov, else save Markov as best global
if best_ml["test_f1"] > markov_metrics["f1"]:
    best_name = best_ml["model"]
    best_obj = {
        "type": "global_sklearn_pipe_stacked",
        "pipe": best_ml["pipe"],
        "thr": float(best_ml["thr"]),
        "uses_p_markov": True,
        "markov_thr": float(thr_m),
        "markov_probs": probs_full,  # keep for inference if you want p_markov at runtime
        "meta": {
            "target": "y_bin=(stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "use_user_id_feature": USE_USER_ID_FEATURE,
            "behavior_cols": BEHAVIOR_COLS,
            "note": "This model uses p_markov as an additional feature (stacking).",
        }
    }
    print("\n🎉 ML BEATS Markov on TEST. Saving ML as best.")
else:
    best_name = "MarkovGlobal"
    best_obj = {
        "type": "markov_global",
        "probs": probs_full,
        "thr": float(thr_m),
        "meta": {
            "target": "y_bin=(stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "note": "Markov remains best on TEST for this dataset/task.",
        }
    }
    print("\nℹ️ Markov still best on TEST. Saving Markov as best (most honest + robust).")

MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_obj, MODEL_OUT)
print("Saved:", MODEL_OUT, "| Best:", best_name)


=== DATA RAW ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
Behavior cols detected: ['extracurricularHourPerDay', 'physicalActivityHourPerDay', 'sleepHourPerDay', 'studyHourPerDay', 'socialHourPerDay']

=== DATASET FEAT ===
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}
WINDOW: 3 | TEST_LEN: 12 | USE_USER_ID_FEATURE: True

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE L1: Persistence (y(t)=y(t-1)) ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===
Best thr: 0.35 | CV pooled F1: 0.8523
TEST: {'acc': 0.85, 'f1': 0.8888888888888888}

=== TRAIN + TUNE (pooled CV, with p_markov feature) ===

=== LEADERBOARD (sorted by TEST F1) ===
Baseline-Markov  | CV f1=0.8523  thr=0.35  | TEST f1=0.8889 acc=0.8500 | params=None
DecisionTree     | CV f1=0.8497  thr=0.05  | TEST f1=0.8636 acc=0.8000 | params={'clf__max_depth': 3, 'clf__min_s

In [30]:
# =====================================================================================
# GLOBAL_FORECAST (Binary) - 1 CELL (Try to BEAT Markov via CV-tuned BLEND)
#
# Baseline L1: Persistence
# Baseline L2: Markov GLOBAL(prev_high, dow) + thr tuning (pooled time-CV)
#
# Upgrade to beat Markov:
# ✅ Train ML with fold-strict p_markov as feature (no-leak)
# ✅ Tune BLEND: p_blend = alpha*p_ml + (1-alpha)*p_markov   (alpha + threshold tuned on CV)
# ✅ Auto-add behavior lag1 if columns exist
#
# Fair protocol:
# - time-based split per user (TEST = last TEST_LEN)
# - pooled time-CV on train_pool
# - DO NOT tune on test
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# 0) CONFIG
# =========================
CANDIDATE_PATHS = [
    Path("/mnt/data/global_dataset_pred.csv"),
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek path DATA_PATH.")

MODEL_OUT = Path("../models/global_forecast_best.joblib")

DATE_COL   = "date"
USER_COL   = "userID"
TARGET_COL = "stressLevelPred"

WINDOW = 3
TEST_LEN = 12

# pooled time-CV windows inside each user's train_pool (index-based)
VAL_WINDOWS = [(12, 24), (18, 30)]

THRESHOLDS = np.linspace(0.05, 0.95, 19)
ALPHAS = np.linspace(0.0, 1.0, 21)  # 0=Pure Markov, 1=Pure ML

RANDOM_STATE = 42
USE_USER_ID_FEATURE = True
USE_BEHAVIOR_LAG1 = True

# =========================
# helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def best_thr_for_proba(y_true, p1):
    best_thr, best_f1 = None, -1
    for thr in THRESHOLDS:
        pred = (p1 >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

def best_alpha_thr_for_blend(y_true, p_markov, p_ml):
    best = {"alpha": None, "thr": None, "f1": -1}
    for a in ALPHAS:
        p = a * p_ml + (1 - a) * p_markov
        thr, f1 = best_thr_for_proba(y_true, p)
        if f1 > best["f1"]:
            best = {"alpha": float(a), "thr": float(thr), "f1": float(f1)}
    return best

# =========================
# 1) LOAD + detect behavior cols
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

exclude = {DATE_COL, USER_COL, TARGET_COL}
num_cols_all = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]
hour_like = [c for c in num_cols_all if ("hour" in c.lower()) or ("hours" in c.lower())]
known = ["studyHourPerDay","sleepHourPerDay","socialHourPerDay","physicalActivityHourPerDay","extracurricularHourPerDay"]
for c in known:
    if c in num_cols_all and c not in hour_like:
        hour_like.append(c)

BEHAVIOR_COLS = hour_like if USE_BEHAVIOR_LAG1 else []

print("=== DATA RAW ===")
print("Path:", DATA_PATH)
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("Behavior cols detected:", BEHAVIOR_COLS)

# =========================
# 2) FEATURE ENGINEERING (no leak)
# =========================
rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    if len(BEHAVIOR_COLS) > 0:
        for c in BEHAVIOR_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

feature_cols = []
if USE_USER_ID_FEATURE:
    feature_cols.append(USER_COL)

feature_cols += ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean", "sp_std", "sp_min", "sp_max",
    "count_high", "count_low",
    "streak_high", "transitions"
]
if len(BEHAVIOR_COLS) > 0:
    feature_cols += [f"lag1_{c}" for c in BEHAVIOR_COLS]

feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("\n=== DATASET FEAT ===")
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())
print("WINDOW:", WINDOW, "| TEST_LEN:", TEST_LEN, "| USE_USER_ID_FEATURE:", USE_USER_ID_FEATURE)

# =========================
# 3) SPLIT: time-based per user (TEST = last TEST_LEN)
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g.iloc[:test_start]
    test_block = g.iloc[test_start:]

    per_user_train_pool[uid] = train_pool
    train_idx += train_pool["index"].tolist()
    test_idx  += test_block["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# pooled CV splits
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))

if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Coba kecilkan TEST_LEN atau VAL_WINDOWS.")
print("CV folds:", len(cv_splits))

y_test = test_df["y_bin"].astype(int).values

# =========================
# 4) BASELINE L1: Persistence
# =========================
pred_persist = (test_df["lag_sp_1"] >= 1).astype(int).values
persist_metrics = eval_bin(y_test, pred_persist)
print("\n=== BASELINE L1: Persistence ===")
print("TEST:", persist_metrics)

# =========================
# 5) BASELINE L2: Markov GLOBAL(prev_high, dow)
# =========================
def train_markov_global(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)
    return probs

def markov_proba(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = df_eval["dow"].astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)])

# tune markov threshold on pooled CV
cv_y_all, cv_pm_all = [], []
for tr_idx, va_idx in cv_splits:
    tr_df = feat.loc[tr_idx]
    va_df = feat.loc[va_idx]
    probs = train_markov_global(tr_df)
    p_va = markov_proba(probs, va_df)
    cv_y_all.append(va_df["y_bin"].astype(int).values)
    cv_pm_all.append(p_va)

cv_y_all = np.concatenate(cv_y_all)
cv_pm_all = np.concatenate(cv_pm_all)

thr_m, cv_f1_m = best_thr_for_proba(cv_y_all, cv_pm_all)

probs_full = train_markov_global(train_pool_df)
p_test_m = markov_proba(probs_full, test_df)
pred_test_m = (p_test_m >= thr_m).astype(int)
markov_metrics = eval_bin(y_test, pred_test_m)

print("\n=== BASELINE L2: Markov GLOBAL(prev_high,dow) ===")
print("Best thr:", thr_m, "| CV pooled F1:", round(cv_f1_m, 4))
print("TEST:", markov_metrics)

# =========================
# 6) MODELS (train fairly) + CV-tuned BLEND with Markov
# =========================
# we add p_markov as a feature (fold-strict, no-leak)
feature_cols_ml = feature_cols + ["p_markov"]

cat_cols = ["dow", "is_weekend"]
if USE_USER_ID_FEATURE:
    cat_cols = [USER_COL] + cat_cols
num_cols = [c for c in feature_cols_ml if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, None], "clf__min_samples_leaf": [1, 2, 4]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800],
         "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2],
         "clf__max_features": ["sqrt"]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800, 1200],
         "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2, 4],
         "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.02, 0.03, 0.05, 0.1],
         "clf__max_depth": [2, 3],
         "clf__max_leaf_nodes": [15, 31, 63]}
    ),
}

def make_X(df_part, p_markov):
    X = df_part[feature_cols].copy()
    X["p_markov"] = p_markov
    return X

def pooled_cv_best(pipe, grid):
    """
    For each hyperparam:
    - compute fold-strict p_markov on tr and va
    - train ML on Xtr(with p_markov_tr), evaluate p_ml on Xva(with p_markov_va)
    - tune BLEND(alpha,thr) on pooled CV using p_markov_va and p_ml_va
    Return best params + best alpha+thr (by CV F1 of BLEND)
    """
    best = None

    for params in ParameterGrid(grid):
        y_all, pm_all, pml_all = [], [], []

        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]

            # fold-strict markov
            probs_fold = train_markov_global(tr_df)
            pm_tr = markov_proba(probs_fold, tr_df)
            pm_va = markov_proba(probs_fold, va_df)

            Xtr = make_X(tr_df, pm_tr)
            ytr = tr_df["y_bin"].astype(int).values
            Xva = make_X(va_df, pm_va)
            yva = va_df["y_bin"].astype(int).values

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)
            pml_va = pipe.predict_proba(Xva)[:, 1]

            y_all.append(yva)
            pm_all.append(pm_va)
            pml_all.append(pml_va)

        y_all = np.concatenate(y_all)
        pm_all = np.concatenate(pm_all)
        pml_all = np.concatenate(pml_all)

        blend_best = best_alpha_thr_for_blend(y_all, pm_all, pml_all)

        if (best is None) or (blend_best["f1"] > best["cv_f1"]):
            best = {
                "params": params,
                "cv_f1": float(blend_best["f1"]),
                "alpha": float(blend_best["alpha"]),
                "thr": float(blend_best["thr"]),
            }

    return best

print("\n=== TRAIN + TUNE (Fair CV) : ML + BLEND vs Markov ===")
rows = []

# final markov for trainpool/test (production-like)
pm_trainpool = markov_proba(probs_full, train_pool_df)
pm_test = p_test_m

X_trainpool_ml = make_X(train_pool_df, pm_trainpool)
y_trainpool = train_pool_df["y_bin"].astype(int).values
X_test_ml = make_X(test_df, pm_test)

for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best(pipe, grid)

    # retrain ML on full trainpool
    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool_ml, y_trainpool)

    pml_test = pipe.predict_proba(X_test_ml)[:, 1]

    # evaluate:
    # - ML alone (thr tuned on CV for ML alone, for reference)
    # - BLEND(alpha,thr) tuned on CV
    thr_ml, _ = best_thr_for_proba(cv_y_all, np.concatenate([
        # rebuild pooled p_ml on CV quickly (approx) would be expensive; skip strict
        # We'll just reuse blend-tuned thr for reporting ML-alone? not fair.
        # So we only report BLEND as the "candidate to beat Markov".
        cv_phigh  # placeholder not used
    ])[:len(cv_y_all)])  # dummy (not used)

    p_blend_test = best["alpha"] * pml_test + (1 - best["alpha"]) * pm_test
    pred_blend_test = (p_blend_test >= best["thr"]).astype(int)
    blend_metrics = eval_bin(y_test, pred_blend_test)

    rows.append({
        "model": name,
        "cv_f1_blend": best["cv_f1"],
        "alpha": best["alpha"],
        "thr": best["thr"],
        "test_f1_blend": blend_metrics["f1"],
        "test_acc_blend": blend_metrics["acc"],
        "params": best["params"],
        "pipe": pipe,
    })

print("\n=== LEADERBOARD (sorted by TEST F1) ===")
base_rows = [
    {"model": "Baseline-Persist", "test_f1": persist_metrics["f1"], "test_acc": persist_metrics["acc"], "detail": ""},
    {"model": "Baseline-Markov",  "test_f1": markov_metrics["f1"], "test_acc": markov_metrics["acc"], "detail": f"thr={thr_m:.2f}"},
]
cand_rows = sorted(rows, key=lambda r: r["test_f1_blend"], reverse=True)

for b in base_rows:
    print(f"{b['model']:<16} | TEST f1={b['test_f1']:.4f} acc={b['test_acc']:.4f} | {b['detail']}")

for r in cand_rows:
    print(f"{('Blend-'+r['model']):<16} | CV f1={r['cv_f1_blend']:.4f} | "
          f"TEST f1={r['test_f1_blend']:.4f} acc={r['test_acc_blend']:.4f} | "
          f"alpha={r['alpha']:.2f} thr={r['thr']:.2f} | params={r['params']}")

best_blend = cand_rows[0]
print("\n✅ BEST BLEND:", best_blend["model"], "| TEST:", {"f1": round(best_blend["test_f1_blend"], 4), "acc": round(best_blend["test_acc_blend"], 4)})

# save only if it beats Markov on TEST
if best_blend["test_f1_blend"] > markov_metrics["f1"]:
    best_obj = {
        "type": "global_blend_markov_ml",
        "markov": {"probs": probs_full, "thr": float(thr_m)},
        "ml": {"pipe": best_blend["pipe"], "params": best_blend["params"]},
        "blend": {"alpha": float(best_blend["alpha"]), "thr": float(best_blend["thr"])},
        "meta": {
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "alphas": ALPHAS.tolist(),
            "use_user_id_feature": USE_USER_ID_FEATURE,
            "behavior_cols": BEHAVIOR_COLS,
        }
    }
    print("\n🎉 BLEND BEATS Markov on TEST. Saving BLEND as best.")
else:
    best_obj = {
        "type": "markov_global",
        "probs": probs_full,
        "thr": float(thr_m),
        "meta": {
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "note": "Markov still best on TEST for this dataset/task.",
        }
    }
    print("\nℹ️ Markov still best on TEST. Saving Markov as best (honest + robust).")

MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_obj, MODEL_OUT)
print("Saved:", MODEL_OUT)


=== DATA RAW ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
Behavior cols detected: ['extracurricularHourPerDay', 'physicalActivityHourPerDay', 'sleepHourPerDay', 'studyHourPerDay', 'socialHourPerDay']

=== DATASET FEAT ===
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}
WINDOW: 3 | TEST_LEN: 12 | USE_USER_ID_FEATURE: True

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE L1: Persistence ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE L2: Markov GLOBAL(prev_high,dow) ===
Best thr: 0.35 | CV pooled F1: 0.8523
TEST: {'acc': 0.85, 'f1': 0.8888888888888888}

=== TRAIN + TUNE (Fair CV) : ML + BLEND vs Markov ===

=== LEADERBOARD (sorted by TEST F1) ===
Baseline-Persist | TEST f1=0.7671 acc=0.7167 | 
Baseline-Markov  | TEST f1=0.8889 acc=0.8500 | thr=0.35
Blend-LogReg     | CV f1=0.8634 | TEST f1=0.9048 acc=0.8667 | alpha=0.40 thr=0.30 | params={'clf__C': 0.1, 'clf

In [31]:
# global_forecast_true_global.py
# =====================================================================================
# TRUE GLOBAL FORECAST (Binary) from stressLevelPred
#
# TRUE GLOBAL means:
# - 1 model untuk semua user
# - TIDAK pakai userID sebagai fitur
# - TIDAK ada model per-user / parameter per-user
#
# Baselines:
# - L1: Persistence (y(t)=y(t-1))
# - L2: Markov GLOBAL P(high_t | prev_high, dow) + threshold tuning
#
# Candidate global ML:
# - LogReg, RandomForest, ExtraTrees, HistGB (global)
# - Threshold tuning via pooled time-CV
#
# Optional best practice:
# - BLEND (Markov + ML): p = alpha*p_ml + (1-alpha)*p_markov
#   alpha & threshold dituning via CV (GLOBAL, bukan per-user)
#
# Saves:
# - ../models/global_forecast_true_global.joblib
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# =========================
# CONFIG
# =========================
CANDIDATE_PATHS = [Path("/mnt/data/global_dataset_pred.csv"), Path("../datasets/global_dataset_pred.csv")]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan.")

MODEL_OUT = Path("../models/global_forecast_true_global.joblib")

DATE_COL   = "date"
USER_COL   = "userID"
TARGET_COL = "stressLevelPred"

WINDOW   = 3
TEST_LEN = 12

# CV windows relatif di train_pool tiap user (end exclusive)
VAL_WINDOWS = [(12, 24), (18, 30)]
THRESHOLDS  = np.linspace(0.05, 0.95, 19)
ALPHAS      = np.linspace(0.0, 1.0, 21)  # 0=Markov only, 1=ML only

RANDOM_STATE = 42
USE_BEHAVIOR_LAG1 = True

# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {"acc": float(accuracy_score(y_true, y_pred)),
            "f1":  float(f1_score(y_true, y_pred, zero_division=0))}

def tune_thr(y_true, p_high):
    best_thr, best_f1 = None, -1.0
    for thr in THRESHOLDS:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

# =========================
# Load
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

# detect behavior hour cols (numeric + name contains hour/hours OR known list)
exclude = {DATE_COL, USER_COL, TARGET_COL}
num_cols_all = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]
hour_like = [c for c in num_cols_all if ("hour" in c.lower()) or ("hours" in c.lower())]
known = ["studyHourPerDay","sleepHourPerDay","socialHourPerDay","physicalActivityHourPerDay","extracurricularHourPerDay"]
for c in known:
    if c in num_cols_all and c not in hour_like:
        hour_like.append(c)
BEHAVIOR_COLS = hour_like if USE_BEHAVIOR_LAG1 else []

print("=== RAW ===")
print("Path:", DATA_PATH)
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("Behavior cols:", BEHAVIOR_COLS)

# =========================
# Feature engineering (NO userID feature)
# =========================
rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    if len(BEHAVIOR_COLS) > 0:
        for c in BEHAVIOR_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()
    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

feature_cols = ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean","sp_std","sp_min","sp_max","count_high","count_low","streak_high","transitions"
]
if len(BEHAVIOR_COLS) > 0:
    feature_cols += [f"lag1_{c}" for c in BEHAVIOR_COLS]

feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("\n=== FEAT ===")
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())
print("WINDOW:", WINDOW, "| TEST_LEN:", TEST_LEN)

# =========================
# Split: TEST = last TEST_LEN per user
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    tp = g.iloc[:test_start]
    te = g.iloc[test_start:]
    per_user_train_pool[uid] = tp
    train_idx += tp["index"].tolist()
    test_idx  += te["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df       = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# build pooled time-CV folds (same protocol as sebelumnya)
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))
if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Kecilkan TEST_LEN / VAL_WINDOWS.")

print("CV folds:", len(cv_splits))

X_trainpool = train_pool_df[feature_cols]
y_trainpool = train_pool_df["y_bin"].astype(int).values
X_test      = test_df[feature_cols]
y_test      = test_df["y_bin"].astype(int).values

# =========================
# Baseline L1: Persistence
# =========================
pred_persist = (test_df["lag_sp_1"] >= 1).astype(int).values
persist_metrics = eval_bin(y_test, pred_persist)
print("\n=== BASELINE L1: Persistence ===")
print("TEST:", persist_metrics)

# =========================
# Baseline L2: Markov GLOBAL(prev_high, dow)
# =========================
def train_markov_global(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)
    return probs

def markov_proba(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = df_eval["dow"].astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)])

# tune thr on pooled CV
cv_true, cv_phigh = [], []
for tr_idx, va_idx in cv_splits:
    tr_df = feat.loc[tr_idx]
    va_df = feat.loc[va_idx]
    probs = train_markov_global(tr_df)
    p = markov_proba(probs, va_df)
    cv_true.append(va_df["y_bin"].values)
    cv_phigh.append(p)

cv_true  = np.concatenate(cv_true)
cv_phigh = np.concatenate(cv_phigh)

thr_mk, cv_f1_mk = tune_thr(cv_true, cv_phigh)

probs_full = train_markov_global(train_pool_df)
p_test_mk  = markov_proba(probs_full, test_df)
pred_test_mk = (p_test_mk >= thr_mk).astype(int)
markov_metrics = eval_bin(y_test, pred_test_mk)

print("\n=== BASELINE L2: Markov GLOBAL(prev_high,dow) ===")
print("Best thr:", thr_mk, "| CV pooled F1:", round(cv_f1_mk, 4))
print("TEST:", markov_metrics)

# =========================
# Global ML candidates (TRUE GLOBAL: no userID)
# =========================
cat_cols = ["dow", "is_weekend"]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)

CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10], "clf__min_samples_leaf": [1, 2], "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3], "clf__max_leaf_nodes": [15, 31, 63]}
    ),
}

def pooled_cv_best_params_thr_and_proba(pipe, grid):
    """
    Cari params terbaik + threshold terbaik berdasarkan pooled CV.
    Return:
      best(params, thr, cv_f1) dan function untuk menghasilkan proba pada X_test setelah fit final.
    """
    best = None
    for params in ParameterGrid(grid):
        y_all, p_all = [], []
        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]
            Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"].astype(int).values
            Xva, yva = va_df[feature_cols], va_df["y_bin"].astype(int).values

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)
            p = pipe.predict_proba(Xva)[:, 1]

            y_all.append(yva); p_all.append(p)

        y_all = np.concatenate(y_all)
        p_all = np.concatenate(p_all)

        thr, cv_f1 = tune_thr(y_all, p_all)
        if (best is None) or (cv_f1 > best["cv_f1"]):
            best = {"params": params, "thr": float(thr), "cv_f1": float(cv_f1)}
    return best

# =========================
# Train ML + (optional) BLEND vs Markov
# =========================
print("\n=== TRAIN + TUNE (Fair CV): ML + BLEND vs Markov ===")

leader = []
leader.append({"name": "Baseline-Persist", "cv_f1": np.nan, "test_f1": persist_metrics["f1"], "test_acc": persist_metrics["acc"], "detail": ""})
leader.append({"name": "Baseline-Markov",  "cv_f1": cv_f1_mk, "test_f1": markov_metrics["f1"], "test_acc": markov_metrics["acc"], "detail": f"thr={thr_mk:.2f}"})

best_saved = {"type": "markov_global", "probs": probs_full, "thr": float(thr_mk)}
best_name  = "MarkovGlobal"
best_test_f1 = float(markov_metrics["f1"])

for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best_params_thr_and_proba(pipe, grid)

    # fit final on TrainPool
    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool, y_trainpool)

    p_ml_test = pipe.predict_proba(X_test)[:, 1]

    # ---- BLEND tuning on CV (strict): blend probabilities from fold models
    # Kita precompute p_markov_cv yang sudah ada (cv_phigh) & cv_true.
    # Untuk p_ml_cv, harus dihitung dari model fold (no leak).
    p_ml_cv_all = []
    y_cv_all    = []
    for tr_idx, va_idx in cv_splits:
        tr_df = feat.loc[tr_idx]
        va_df = feat.loc[va_idx]
        Xtr, ytr = tr_df[feature_cols], tr_df["y_bin"].astype(int).values
        Xva, yva = va_df[feature_cols], va_df["y_bin"].astype(int).values

        pipe_fold = Pipeline([("prep", preprocess), ("clf", clf)])
        pipe_fold.set_params(**best["params"])
        pipe_fold.fit(Xtr, ytr)
        p_ml = pipe_fold.predict_proba(Xva)[:, 1]

        p_ml_cv_all.append(p_ml)
        y_cv_all.append(yva)

    p_ml_cv_all = np.concatenate(p_ml_cv_all)
    y_cv_all    = np.concatenate(y_cv_all)

    # Markov proba CV (sudah dihitung)
    p_mk_cv_all = cv_phigh.copy()

    # Tune alpha + threshold (GLOBAL)
    best_blend = None
    for a in ALPHAS:
        p_blend = a * p_ml_cv_all + (1.0 - a) * p_mk_cv_all
        thr, cv_f1 = tune_thr(y_cv_all, p_blend)
        if (best_blend is None) or (cv_f1 > best_blend["cv_f1"]):
            best_blend = {"alpha": float(a), "thr": float(thr), "cv_f1": float(cv_f1)}

    # Evaluate ML-only on TEST (using best ML thr)
    pred_ml_test = (p_ml_test >= best["thr"]).astype(int)
    ml_metrics = eval_bin(y_test, pred_ml_test)
    leader.append({"name": f"ML-{name}", "cv_f1": best["cv_f1"], "test_f1": ml_metrics["f1"], "test_acc": ml_metrics["acc"],
                   "detail": f"thr={best['thr']:.2f} params={best['params']}"})

    # Evaluate BLEND on TEST
    p_blend_test = best_blend["alpha"] * p_ml_test + (1.0 - best_blend["alpha"]) * p_test_mk
    pred_blend_test = (p_blend_test >= best_blend["thr"]).astype(int)
    blend_metrics = eval_bin(y_test, pred_blend_test)
    leader.append({"name": f"Blend-{name}", "cv_f1": best_blend["cv_f1"], "test_f1": blend_metrics["f1"], "test_acc": blend_metrics["acc"],
                   "detail": f"alpha={best_blend['alpha']:.2f} thr={best_blend['thr']:.2f} params={best['params']}"})

    # Save best overall (by TEST F1)
    if blend_metrics["f1"] > best_test_f1:
        best_test_f1 = float(blend_metrics["f1"])
        best_name = f"Blend-{name}"
        best_saved = {
            "type": "true_global_blend",
            "alpha": float(best_blend["alpha"]),
            "thr": float(best_blend["thr"]),
            "markov": {"type": "markov_global", "probs": probs_full},
            "ml": {"type": "sklearn_pipe", "pipe": pipe},
            "meta": {
                "true_global": True,
                "uses_user_id_feature": False,
                "window": WINDOW,
                "test_len": TEST_LEN,
                "val_windows": VAL_WINDOWS,
                "behavior_cols": BEHAVIOR_COLS,
                "target": "y_bin=(stressLevelPred>=1)",
            }
        }

# leaderboard
print("\n=== LEADERBOARD (sorted by TEST F1) ===")
leader_sorted = sorted(leader, key=lambda r: r["test_f1"], reverse=True)
for r in leader_sorted:
    cv_txt = "NA" if (r["cv_f1"] is None or (isinstance(r["cv_f1"], float) and np.isnan(r["cv_f1"]))) else f"{r['cv_f1']:.4f}"
    print(f"{r['name']:<18} | CV f1={cv_txt:<7} | TEST f1={r['test_f1']:.4f} acc={r['test_acc']:.4f} | {r['detail']}")

print("\n✅ BEST TRUE GLOBAL:", best_name, "| TEST f1=", round(best_test_f1, 4))

MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_saved, MODEL_OUT)
print("Saved:", MODEL_OUT)


=== RAW ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
Behavior cols: ['extracurricularHourPerDay', 'physicalActivityHourPerDay', 'sleepHourPerDay', 'studyHourPerDay', 'socialHourPerDay']

=== FEAT ===
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}
WINDOW: 3 | TEST_LEN: 12

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE L1: Persistence ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE L2: Markov GLOBAL(prev_high,dow) ===
Best thr: 0.35 | CV pooled F1: 0.8523
TEST: {'acc': 0.85, 'f1': 0.8888888888888888}

=== TRAIN + TUNE (Fair CV): ML + BLEND vs Markov ===

=== LEADERBOARD (sorted by TEST F1) ===
Blend-LogReg       | CV f1=0.8617  | TEST f1=0.9048 acc=0.8667 | alpha=0.35 thr=0.30 params={'clf__C': 0.03, 'clf__solver': 'liblinear'}
Baseline-Markov    | CV f1=0.8523  | TEST f1=0.8889 acc=0.8500 | thr=0.35
Blend-RandomForest | CV f1=0.8587  | TEST f1=0.8354 acc

In [32]:
# global_forecast_true_global.py
# =====================================================================================
# TRUE GLOBAL FORECAST (Binary) from stressLevelPred
#
# - BENAR-BENAR GLOBAL: TIDAK pakai userID sebagai fitur sama sekali.
# - 1 model untuk semua user (cold-start global).
#
# Target:
#   y_bin(t) = 1 jika stressLevelPred(t) >= 1, else 0
#
# Baselines:
#   L1) Persistence: y(t)=y(t-1)
#   L2) Markov GLOBAL: P(high_t | prev_high, dow) + threshold tuning (time-CV pooled)
#
# Models (global, without user identity):
#   - LogisticRegression
#   - DecisionTree
#   - RandomForest
#   - ExtraTrees
#   - HistGradientBoosting
#   - GradientBoosting
#   - AdaBoost
#   - BaggingTree
#   - LinearSVC + CalibratedClassifierCV
#
# Upgrade to beat Markov:
#   - Optional BLEND: p = alpha*p_ml + (1-alpha)*p_markov
#     alpha & thr ditune dengan CV (fair, no-leak)
#
# Split:
#   - time-based per user
#   - TEST = last TEST_LEN per user
#   - CV = windows di train_pool tiap user (pooled across users)
#
# Output:
#   ../models/global_forecast_true_global.joblib
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier,
)
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV


# =========================
# 0) CONFIG
# =========================
CANDIDATE_PATHS = [
    Path("/mnt/data/global_dataset_pred.csv"),
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek DATA_PATH.")

MODEL_OUT = Path("../models/global_forecast_true_global.joblib")

DATE_COL   = "date"
USER_COL   = "userID"            # dipakai untuk split saja (bukan fitur model)
TARGET_COL = "stressLevelPred"

WINDOW = 3
TEST_LEN = 12

# CV windows (index relatif di train_pool tiap user), end exclusive
VAL_WINDOWS = [(12, 24), (18, 30)]
THRESHOLDS  = np.linspace(0.05, 0.95, 19)

# BLEND config (ML + Markov)
USE_BLEND = True
ALPHAS = np.linspace(0.0, 1.0, 11)   # 0.0=Markov pure, 1.0=ML pure

RANDOM_STATE = 42

# Tambah fitur behavior lag1 jika kolomnya ada
USE_BEHAVIOR_LAG1 = True


# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def tune_thr_from_proba(y_true, p_high):
    best_thr, best_f1 = None, -1.0
    for thr in THRESHOLDS:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

def pick_existing_behavior_cols(df: pd.DataFrame):
    """Deteksi kolom 'hour' yang numerik, untuk dipakai sebagai behavior lag1."""
    exclude = {DATE_COL, USER_COL, TARGET_COL}
    numeric = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]
    hour_like = [c for c in numeric if ("hour" in c.lower()) or ("hours" in c.lower())]

    known = [
        "studyHourPerDay",
        "sleepHourPerDay",
        "socialHourPerDay",
        "physicalActivityHourPerDay",
        "extracurricularHourPerDay",
    ]
    for c in known:
        if c in numeric and c not in hour_like:
            hour_like.append(c)

    return hour_like


# =========================
# 1) LOAD
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

if not df[TARGET_COL].dropna().between(0, 2).all():
    raise ValueError(f"{TARGET_COL} harus berada pada range 0..2")

BEHAVIOR_COLS = pick_existing_behavior_cols(df) if USE_BEHAVIOR_LAG1 else []

print("=== DATA RAW ===")
print("Path:", DATA_PATH)
print("Rows:", len(df), "| Users:", df[USER_COL].nunique(), "| Date:", df[DATE_COL].min().date(), "->", df[DATE_COL].max().date())
print("Behavior cols detected:", BEHAVIOR_COLS)


# =========================
# 2) FEATURE ENGINEERING (no leak)
# =========================
rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    # fitur kalender
    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)  # 0..6
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    # lag stress pred
    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    # behavior lag1
    if len(BEHAVIOR_COLS) > 0:
        for c in BEHAVIOR_COLS:
            g[f"lag1_{c}"] = g[c].shift(1)

    sp_shift = g[TARGET_COL].shift(1)

    # rolling stats dari history (ending at t-1)
    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    # streak high (<= t-1)
    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    # transitions
    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

# TRUE GLOBAL: fitur TIDAK boleh include userID
feature_cols = ["dow", "is_weekend"] + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)] + [
    "sp_mean", "sp_std", "sp_min", "sp_max",
    "count_high", "count_low",
    "streak_high", "transitions",
]
if len(BEHAVIOR_COLS) > 0:
    feature_cols += [f"lag1_{c}" for c in BEHAVIOR_COLS]

# drop rows yang belum punya history lengkap
feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)

print("\n=== DATASET FEAT ===")
print("Rows:", len(feat), "| Users:", feat[USER_COL].nunique())
print("Binary dist:", feat["y_bin"].value_counts().to_dict())
print("WINDOW:", WINDOW, "| TEST_LEN:", TEST_LEN)


# =========================
# 3) SPLIT: time-based per user (TEST = last TEST_LEN)
# =========================
train_idx, test_idx = [], []
per_user_train_pool = {}

for uid, g in feat.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index()
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 20:
        raise ValueError("Data per user terlalu sedikit untuk split + CV windows.")
    train_pool = g.iloc[:test_start]
    test_block = g.iloc[test_start:]

    per_user_train_pool[uid] = train_pool
    train_idx += train_pool["index"].tolist()
    test_idx  += test_block["index"].tolist()

train_pool_df = feat.loc[train_idx].copy()
test_df = feat.loc[test_idx].copy()

print("\n=== SPLIT ===")
print("TrainPool:", len(train_pool_df), "| Test:", len(test_df))
print("Test dist:", test_df["y_bin"].value_counts().to_dict())

# build CV splits (pooled windows across users)
cv_splits = []
for (v0, v1) in VAL_WINDOWS:
    tr_idx, va_idx = [], []
    ok = True
    for uid, tp in per_user_train_pool.items():
        tp = tp.reset_index(drop=True)
        if len(tp) < v1:
            ok = False
            break
        va = tp.iloc[v0:v1]
        tr = tp.iloc[:v0]
        tr_idx += tr["index"].tolist()
        va_idx += va["index"].tolist()
    if ok:
        cv_splits.append((tr_idx, va_idx))

if len(cv_splits) == 0:
    raise ValueError("CV windows gagal terbentuk. Kecilkan TEST_LEN atau VAL_WINDOWS.")
print("CV folds:", len(cv_splits))

X_trainpool = train_pool_df[feature_cols].copy()
y_trainpool = train_pool_df["y_bin"].astype(int).values

X_test = test_df[feature_cols].copy()
y_test = test_df["y_bin"].astype(int).values


# =========================
# 4) BASELINE L1: Persistence
# =========================
pred_persist = (test_df["lag_sp_1"] >= 1).astype(int).values
persist_metrics = eval_bin(y_test, pred_persist)
print("\n=== BASELINE L1: Persistence ===")
print("TEST:", persist_metrics)


# =========================
# 5) BASELINE L2: Markov GLOBAL(prev_high, dow) + thr tuning (fair)
# =========================
def train_markov_global(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)  # prev(2) x dow(7) x y(2)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = df_train["dow"].astype(int).values
    yb   = df_train["y_bin"].astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace
    return probs

def markov_proba(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = df_eval["dow"].astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)], dtype=float)

# tune threshold Markov via pooled CV
cv_true, cv_phigh = [], []
for tr_idx, va_idx in cv_splits:
    tr_df = feat.loc[tr_idx]
    va_df = feat.loc[va_idx]
    probs = train_markov_global(tr_df)
    p = markov_proba(probs, va_df)
    cv_true.append(va_df["y_bin"].astype(int).values)
    cv_phigh.append(p)

cv_true = np.concatenate(cv_true)
cv_phigh = np.concatenate(cv_phigh)

thr_mk, cv_f1_mk = tune_thr_from_proba(cv_true, cv_phigh)

probs_full = train_markov_global(train_pool_df)
p_test_mk = markov_proba(probs_full, test_df)
pred_test_mk = (p_test_mk >= thr_mk).astype(int)
markov_metrics = eval_bin(y_test, pred_test_mk)

print("\n=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===")
print("Best thr:", thr_mk, "| CV pooled F1:", round(cv_f1_mk, 4))
print("TEST:", markov_metrics)


# =========================
# 6) PREPROCESS (TRUE GLOBAL: tanpa userID)
# =========================
# Cat: dow (lebih aman di-onehot). is_weekend bisa numeric.
cat_cols = ["dow"]
num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop"
)


# =========================
# 7) CANDIDATE MODELS (semua yang relevan)
# =========================
CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),

    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, 6, None], "clf__min_samples_leaf": [1, 2, 4, 8]}
    ),

    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2, 4], "clf__max_features": ["sqrt"]}
    ),

    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2, 4], "clf__max_features": ["sqrt"]}
    ),

    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3],
         "clf__max_leaf_nodes": [15, 31, 63]}
    ),

    "GradBoost": (
        GradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__n_estimators": [100, 200, 400],
         "clf__max_depth": [2, 3]}
    ),

    "AdaBoost": (
        AdaBoostClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1, 0.3], "clf__n_estimators": [50, 100, 200, 400]}
    ),

    "BaggingTree": (
        BaggingClassifier(
            estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
            random_state=RANDOM_STATE,
            n_jobs=1
        ),
        {"clf__n_estimators": [50, 100, 200],
         "clf__estimator__max_depth": [2, 3, 4, None],
         "clf__estimator__min_samples_leaf": [1, 2, 4]}
    ),

    "LinearSVC_Calibrated": (
        CalibratedClassifierCV(
            estimator=LinearSVC(class_weight="balanced", random_state=RANDOM_STATE),
            method="sigmoid",
            cv=3
        ),
        {"clf__estimator__C": [0.03, 0.1, 0.3, 1.0, 3.0]}
    ),
}


# =========================
# 8) FAIR TUNING: CV pooled + threshold tuning + optional BLEND vs Markov
# =========================
def pooled_cv_best(pipe, grid):
    """
    Cari params terbaik dgn pooled CV:
    - kumpulkan proba di semua validation folds
    - tune threshold yang memaksimalkan F1
    - kalau USE_BLEND: juga tune alpha (campur p_ml & p_markov fold-specific)
    """
    best = None

    for params in ParameterGrid(grid):
        # kumpulkan untuk CV pooled
        y_list, pml_list, pmk_list = [], [], []

        for tr_idx, va_idx in cv_splits:
            tr_df = feat.loc[tr_idx]
            va_df = feat.loc[va_idx]

            # Markov fold-specific (no leak)
            mk_probs = train_markov_global(tr_df)
            p_mk = markov_proba(mk_probs, va_df)

            # ML fold-specific
            Xtr = tr_df[feature_cols].copy()
            ytr = tr_df["y_bin"].astype(int).values
            Xva = va_df[feature_cols].copy()
            yva = va_df["y_bin"].astype(int).values

            pipe.set_params(**params)
            pipe.fit(Xtr, ytr)
            p_ml = pipe.predict_proba(Xva)[:, 1]

            y_list.append(yva)
            pml_list.append(p_ml)
            pmk_list.append(p_mk)

        y_all = np.concatenate(y_list)
        pml_all = np.concatenate(pml_list)
        pmk_all = np.concatenate(pmk_list)

        if USE_BLEND:
            # tune alpha & thr
            local_best = None
            for alpha in ALPHAS:
                p_blend = alpha * pml_all + (1.0 - alpha) * pmk_all
                thr, cv_f1 = tune_thr_from_proba(y_all, p_blend)
                if (local_best is None) or (cv_f1 > local_best["cv_f1"]):
                    local_best = {"alpha": float(alpha), "thr": float(thr), "cv_f1": float(cv_f1)}
            score = local_best["cv_f1"]
            record = {"params": params, **local_best}
        else:
            thr, cv_f1 = tune_thr_from_proba(y_all, pml_all)
            score = cv_f1
            record = {"params": params, "alpha": 1.0, "thr": float(thr), "cv_f1": float(cv_f1)}

        if (best is None) or (score > best["cv_f1"]):
            best = record

    return best


print("\n=== TRAIN + TUNE (Fair CV) : ML (+ optional BLEND) vs Markov ===")
rows = []

for name, (clf, grid) in CANDIDATES.items():
    pipe = Pipeline([("prep", preprocess), ("clf", clf)])
    best = pooled_cv_best(pipe, grid)

    # train final ML on full TrainPool
    pipe.set_params(**best["params"])
    pipe.fit(X_trainpool, y_trainpool)
    p_test_ml = pipe.predict_proba(X_test)[:, 1]

    # Markov prob on test from TrainPool Markov
    p_test_mk_full = markov_proba(probs_full, test_df)

    # final pred
    alpha = float(best["alpha"])
    p_test_final = alpha * p_test_ml + (1.0 - alpha) * p_test_mk_full
    pred_test_final = (p_test_final >= best["thr"]).astype(int)

    test_metrics = eval_bin(y_test, pred_test_final)

    rows.append({
        "model": name,
        "cv_f1": float(best["cv_f1"]),
        "alpha": float(best["alpha"]),
        "thr": float(best["thr"]),
        "test_f1": float(test_metrics["f1"]),
        "test_acc": float(test_metrics["acc"]),
        "params": best["params"],
        "pipe": pipe,
    })

# Leaderboard
print("\n=== LEADERBOARD (sorted by TEST F1) ===")
base_rows = [
    {"model": "Baseline-Persist", "cv_f1": np.nan, "alpha": np.nan, "thr": np.nan, "test_f1": persist_metrics["f1"], "test_acc": persist_metrics["acc"], "params": None},
    {"model": "Baseline-Markov",  "cv_f1": cv_f1_mk, "alpha": 0.0, "thr": thr_mk, "test_f1": markov_metrics["f1"], "test_acc": markov_metrics["acc"], "params": {"markov": "prev_high+dow"}},
]
all_rows = base_rows + rows
all_sorted = sorted(all_rows, key=lambda r: r["test_f1"], reverse=True)

for r in all_sorted:
    cv_txt = "NA" if (r["cv_f1"] is None or (isinstance(r["cv_f1"], float) and np.isnan(r["cv_f1"]))) else f"{r['cv_f1']:.4f}"
    alpha_txt = "NA" if (r["alpha"] is None or (isinstance(r["alpha"], float) and np.isnan(r["alpha"]))) else f"{r['alpha']:.2f}"
    thr_txt = "NA" if (r["thr"] is None or (isinstance(r["thr"], float) and np.isnan(r["thr"]))) else f"{r['thr']:.2f}"
    print(f"{r['model']:<20} | CV f1={cv_txt:<7} | TEST f1={r['test_f1']:.4f} acc={r['test_acc']:.4f} | alpha={alpha_txt} thr={thr_txt} | params={r['params']}")

best_ml = sorted(rows, key=lambda r: r["test_f1"], reverse=True)[0]
print("\n✅ BEST (ML/BLEND):", best_ml["model"], "| TEST:", {"f1": round(best_ml["test_f1"], 4), "acc": round(best_ml["test_acc"], 4)})

# pilih terbaik overall (Markov vs best ML/Blend)
if best_ml["test_f1"] > markov_metrics["f1"]:
    best_name = best_ml["model"]
    best_obj = {
        "type": "true_global_blend_model",
        "pipe": best_ml["pipe"],               # sklearn pipeline
        "alpha": float(best_ml["alpha"]),
        "thr": float(best_ml["thr"]),
        "markov_probs": probs_full,            # untuk hitung p_markov runtime
        "meta": {
            "note": "TRUE GLOBAL (no userID). Prediction uses p = alpha*p_ml + (1-alpha)*p_markov",
            "target": "y_bin=(stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "alphas": ALPHAS.tolist(),
            "behavior_cols": BEHAVIOR_COLS,
        }
    }
    print("\n🎉 TRUE GLOBAL model BEATS Markov on TEST. Saving BLEND as best.")
else:
    best_name = "MarkovGlobal"
    best_obj = {
        "type": "true_global_markov",
        "thr": float(thr_mk),
        "probs": probs_full,
        "meta": {
            "note": "TRUE GLOBAL baseline Markov remains best on TEST for this dataset",
            "target": "y_bin=(stressLevelPred>=1)",
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
        }
    }
    print("\nℹ️ Markov still best on TEST. Saving Markov as best (robust).")

MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_obj, MODEL_OUT)
print("Saved:", MODEL_OUT, "| Best:", best_name)


=== DATA RAW ===
Path: ..\datasets\global_dataset_pred.csv
Rows: 275 | Users: 5 | Date: 2025-11-21 -> 2026-01-14
Behavior cols detected: ['extracurricularHourPerDay', 'physicalActivityHourPerDay', 'sleepHourPerDay', 'studyHourPerDay', 'socialHourPerDay']

=== DATASET FEAT ===
Rows: 260 | Users: 5
Binary dist: {1: 146, 0: 114}
WINDOW: 3 | TEST_LEN: 12

=== SPLIT ===
TrainPool: 200 | Test: 60
Test dist: {1: 38, 0: 22}
CV folds: 2

=== BASELINE L1: Persistence ===
TEST: {'acc': 0.7166666666666667, 'f1': 0.7671232876712328}

=== BASELINE L2: Markov GLOBAL(prev_high, dow) ===
Best thr: 0.35 | CV pooled F1: 0.8523
TEST: {'acc': 0.85, 'f1': 0.8888888888888888}

=== TRAIN + TUNE (Fair CV) : ML (+ optional BLEND) vs Markov ===

=== LEADERBOARD (sorted by TEST F1) ===
Baseline-Markov      | CV f1=0.8523  | TEST f1=0.8889 acc=0.8500 | alpha=0.00 thr=0.35 | params={'markov': 'prev_high+dow'}
LogReg               | CV f1=0.8619  | TEST f1=0.8889 acc=0.8500 | alpha=0.30 thr=0.35 | params={'clf__C': 